In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np 
import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

In [2]:
# Import our input dataset
crime_df = pd.read_csv('All_data_cleaned.csv')
crime_df.head()

crime_df.dtypes

Year                                                            int64
City                                                           object
Population 25 years and over - Less than 9th grade              int64
Percent Population 25 years and over - Less than 9th grade    float64
Population 25 years and over - 9th to12th (No Diploma)          int64
                                                               ...   
Percent 15 to 24                                              float64
Percent Uneducated                                            float64
Percent Higher Education                                      float64
Sum Uneducated                                                  int64
Sum Higher Education                                            int64
Length: 61, dtype: object

In [3]:
# One-hot encode the 'City' column
crime_encoded = pd.get_dummies(crime_df, columns=['City'])

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the 'City' column
crime_df['City_encoded'] = label_encoder.fit_transform(crime_df['City'])

crime_df

,Year,City,Population 25 years and over - Less than 9th grade,Percent Population 25 years and over - Less than 9th grade,Population 25 years and over - 9th to12th (No Diploma),Percent Population 25 years and over - 9th to12th (No Diploma),Population 25 years and over - High School Graduate (and equivalent),Percent Population 25 years and over - High School Graduate (and equivalent),"Population 25 years and over - Some college, no degree","Percent Population 25 years and over - Some college, no degree",...,Percent Home Occupied,Percent Renter Occupied,Percent 25+,Percent 14-,Percent 15 to 24,Percent Uneducated,Percent Higher Education,Sum Uneducated,Sum Higher Education,City_encoded
0,2010,Alameda,2752,5.50,2234,4.50,8546,17.00,10009,19.90,...,46.920853,53.079147,65.8,19.8,12.3,10.00,53.10,4986,26639,0
1,2010,Alhambra,6236,10.10,4575,7.40,13358,21.70,12091,19.60,...,38.053381,61.946619,71.1,12.9,13.1,17.50,41.10,10811,25307,1
2,2010,Anaheim,32561,15.50,22329,10.70,50982,24.30,39407,18.80,...,48.388510,51.611490,61.0,21.2,16.7,26.20,30.70,54890,64362,2
3,2010,Antioch,4874,7.70,4633,7.30,17092,26.90,20113,31.70,...,62.767194,37.232806,60.7,22.9,15.5,15.00,26.50,9507,16815,3
4,2010,Apple Valley,639,1.50,3353,7.70,14524,33.40,11690,26.90,...,70.690377,29.309623,60.4,23.4,13.9,9.20,30.50,3992,13303,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1160,2019,Vista,10080,15.05,6638,9.91,13245,19.77,16027,23.93,...,45.724859,54.275141,64.4,19.9,14.1,24.96,31.34,16718,20990,129
1161,2019,West Covina,5308,7.09,4530,6.05,20919,27.95,16083,21.49,...,57.975360,42.024640,69.5,16.9,11.9,13.14,37.41,9838,28003,131
1162,2019,Westminster,8307,13.03,6607,10.36,15641,24.53,11273,17.68,...,47.216869,52.783131,68.0,18.2,11.5,23.39,34.41,14914,21943,132
1163,2019,Whittier,3021,5.44,3921,7.06,16232,29.24,12079,21.76,...,60.928433,39.071567,63.3,21.0,13.9,12.50,36.49,6942,20258,133


In [4]:
# Fix the seed for random number generation
seed_value = 8
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Remove crime and city target from features data
X = crime_df.copy()
X.drop(columns=["Crime_Rate_per_100k", 
                "City",
                "Violent Crimes Sum"
               ], axis=1, inplace=True)
y = crime_df['Crime_Rate_per_100k']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 3: Split Data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.1, random_state=8)

In [5]:
# build model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)  # No activation for regression
])

# compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# train model
history = model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_split=0.15)

# evaluate the model
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

#predict
predictions = model.predict(X_test)

Epoch 1/1000


C:\Users\EJ-PC\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 220159.8125 - val_loss: 232234.9688
Epoch 2/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 214189.2656 - val_loss: 219677.5000
Epoch 3/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 199125.0469 - val_loss: 192781.8594
Epoch 4/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 169424.7812 - val_loss: 153820.2344
Epoch 5/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 129395.0078 - val_loss: 110746.7500
Epoch 6/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 92973.0625 - val_loss: 81866.8984
Epoch 7/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 75710.7500 - val_loss: 70337.3203
Epoch 8/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 70191.7344 - val_loss: 64703.2305
Epoch 9/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 66354.8984 - val_loss: 60859.9570
Epoch 10/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 63148.2500 - val_loss: 57913.4180
Epoch 11/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 6

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 29493.4707 - val_loss: 31147.4121
Epoch 65/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 29239.6719 - val_loss: 30985.3691
Epoch 66/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 29002.9766 - val_loss: 30857.9805
Epoch 67/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 28771.8105 - val_loss: 30758.6621
Epoch 68/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 28539.6934 - val_loss: 30613.2500
Epoch 69/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 28302.9238 - val_loss: 30474.4102
Epoch 70/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 28073.0332 - val_loss: 30348.9434
Epoch 71/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 27832.5879 - val_loss: 30270.6367
Epoch 72/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 27606.1484 - val_loss: 30150.5898
Epoch 73/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 27370.5762 - val_loss: 29960.8750
Epoch 74/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 271

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 15275.3779 - val_loss: 23131.0742
Epoch 128/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 15100.0586 - val_loss: 22939.0898
Epoch 129/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 14921.8086 - val_loss: 22835.8340
Epoch 130/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 14730.5205 - val_loss: 22729.9785
Epoch 131/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 14580.6104 - val_loss: 22610.0020
Epoch 132/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 14393.4570 - val_loss: 22519.3125
Epoch 133/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 14228.6338 - val_loss: 22383.1465
Epoch 134/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 14073.4619 - val_loss: 22378.5156
Epoch 135/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 13895.9170 - val_loss: 22271.1680
Epoch 136/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 13749.9658 - val_loss: 22243.9180
Epoch 137/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step -

...

Epoch 696/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 629.4134 - val_loss: 25178.1816
Epoch 697/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 583.9717 - val_loss: 25350.0254
Epoch 698/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 617.3992 - val_loss: 25194.0742
Epoch 699/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 578.2355 - val_loss: 25371.9629
Epoch 700/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 603.2261 - val_loss: 25233.0879
Epoch 701/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 577.7198 - val_loss: 25336.5977
Epoch 702/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 574.2580 - val_loss: 25380.8477
Epoch 703/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 574.2878 - val_loss: 25326.7188
Epoch 704/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 559.1187 - val_loss: 25388.9062
Epoch 705/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 573.5446 - val_loss: 25341.1504
Epoch 706/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 458.0383 - val_loss: 26255.6992
Epoch 761/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 412.7125 - val_loss: 26479.5527
Epoch 762/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 457.6684 - val_loss: 26406.1875
Epoch 763/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 422.9797 - val_loss: 26596.6367
Epoch 764/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 461.3939 - val_loss: 26508.1621
Epoch 765/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 431.2374 - val_loss: 26543.1348
Epoch 766/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 457.6639 - val_loss: 26447.9258
Epoch 767/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 413.0385 - val_loss: 26577.4434
Epoch 768/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 463.1451 - val_loss: 26486.4727
Epoch 769/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 430.8039 - val_loss: 26528.0312
Epoch 770/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 432.5244 - va

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 349.8093 - val_loss: 27170.3125
Epoch 825/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 344.1866 - val_loss: 27226.7441
Epoch 826/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 345.5498 - val_loss: 27122.1973
Epoch 827/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 332.2007 - val_loss: 27140.8965
Epoch 828/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 339.4282 - val_loss: 27219.3086
Epoch 829/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 330.2219 - val_loss: 27242.9902
Epoch 830/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 341.1563 - val_loss: 27311.3945
Epoch 831/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 327.0164 - val_loss: 27213.5898
Epoch 832/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 320.5042 - val_loss: 27225.8711
Epoch 833/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 326.1962 - val_loss: 27217.1758
Epoch 834/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 315.9898 - va

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 347.9998 - val_loss: 26555.4746
Epoch 889/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 365.9950 - val_loss: 26456.7402
Epoch 890/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 353.8341 - val_loss: 26365.2090
Epoch 891/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 368.7873 - val_loss: 26379.0371
Epoch 892/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 373.1602 - val_loss: 26249.2383
Epoch 893/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 369.0885 - val_loss: 26198.7754
Epoch 894/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 386.4906 - val_loss: 26113.4492
Epoch 895/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 379.0771 - val_loss: 26128.2090
Epoch 896/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 405.9471 - val_loss: 25849.8691
Epoch 897/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 378.8235 - val_loss: 26149.0039
Epoch 898/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 456.7093 - va

Epoch 952/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 808.4896 - val_loss: 23843.2402
Epoch 953/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 792.6819 - val_loss: 23915.5117
Epoch 954/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 782.3024 - val_loss: 23945.0273
Epoch 955/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 775.1335 - val_loss: 23997.5059
Epoch 956/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 765.5756 - val_loss: 24051.4258
Epoch 957/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 757.2414 - val_loss: 24086.9023
Epoch 958/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 747.4750 - val_loss: 24130.7852
Epoch 959/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 740.3379 - val_loss: 24164.0430
Epoch 960/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 732.6537 - val_loss: 24208.9844
Epoch 961/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 724.8267 - val_loss: 24229.8691
Epoch 962/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss

In [6]:
if predictions.ndim != 1:
    predictions = predictions.flatten()

# Check if 'y_test' is not 1-dimensional and reshape if necessary
if y_test.ndim != 1:
    y_test = y_test.flatten()
    
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
print(results)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(mae, 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

     Prediction      Actual
0    140.377411  119.793729
1    423.789185  359.563884
2    287.398407  168.934525
3    334.234314  355.505368
4    459.209747  433.151080
..          ...         ...
112  571.628357  487.957347
113  291.729767  216.123044
114  321.697601  425.953734
115  199.423340  436.229287
116  445.682251  388.424770

[117 rows x 2 columns]
Mean Absolute Error (MAE): 89.32138804135373
Mean Squared Error (MSE): 14357.810248065185
Root Mean Squared Error (RMSE): 119.82408041819134
R-squared (R²): 0.7458280367897643
Mean Absolute Error: 89.32 degrees.
Accuracy: 66.46 %.


C:\Users\EJ-PC\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [7]:
# build model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)  # No activation for regression
])

# compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# train model
history = model.fit(X_train, y_train, epochs=500, batch_size=32, validation_split=0.15)

# evaluate the model
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

#predict
predictions = model.predict(X_test)

Epoch 1/500


C:\Users\EJ-PC\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 220357.4531 - val_loss: 233181.0312
Epoch 2/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 215408.8750 - val_loss: 222596.0469
Epoch 3/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 202447.5469 - val_loss: 197916.0625
Epoch 4/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 174444.0156 - val_loss: 158995.0312
Epoch 5/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 134462.9062 - val_loss: 117942.9141
Epoch 6/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 98844.7812 - val_loss: 89667.9375
Epoch 7/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 80972.0078 - val_loss: 76210.1172
Epoch 8/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 73872.4219 - val_loss: 69030.7656
Epoch 9/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 69164.9922 - val_loss: 64601.0430
Epoch 10/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 65695.1250 - val_loss: 61509.3359
Epoch 11/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 62986.6367 

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 27274.3633 - val_loss: 30463.6211
Epoch 66/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 27009.1035 - val_loss: 30318.5371
Epoch 67/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 26717.2207 - val_loss: 30115.0059
Epoch 68/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 26434.3672 - val_loss: 29991.0156
Epoch 69/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 26187.3184 - val_loss: 29806.5879
Epoch 70/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 25907.8379 - val_loss: 29641.5410
Epoch 71/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 25639.7012 - val_loss: 29470.8008
Epoch 72/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 25381.5059 - val_loss: 29315.8750
Epoch 73/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 25128.4062 - val_loss: 29109.4277
Epoch 74/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 24874.1641 - val_loss: 29049.4102
Epoch 75/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 24645.1387 - 

Epoch 129/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 16866.0488 - val_loss: 24839.8477
Epoch 130/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 16762.7207 - val_loss: 24773.9102
Epoch 131/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 16672.2695 - val_loss: 24738.0469
Epoch 132/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 16573.7695 - val_loss: 24656.6777
Epoch 133/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 16495.4707 - val_loss: 24596.5566
Epoch 134/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 16390.6348 - val_loss: 24499.4570
Epoch 135/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 16287.1387 - val_loss: 24522.6602
Epoch 136/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 16215.8398 - val_loss: 24390.9648
Epoch 137/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 16129.4629 - val_loss: 24346.5254
Epoch 138/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 16032.6230 - val_loss: 24222.5996
Epoch 139/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/st

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 11789.1914 - val_loss: 20588.0527
Epoch 193/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 11706.3262 - val_loss: 20549.5527
Epoch 194/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 11643.1123 - val_loss: 20504.4121
Epoch 195/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 11574.2100 - val_loss: 20430.3867
Epoch 196/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 11500.7832 - val_loss: 20372.2793
Epoch 197/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 11429.2363 - val_loss: 20308.3652
Epoch 198/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 11358.4639 - val_loss: 20273.6289
Epoch 199/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 11286.8037 - val_loss: 20244.8984
Epoch 200/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 11207.6152 - val_loss: 20175.5195
Epoch 201/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 11151.5830 - val_loss: 20150.3086
Epoch 202/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 110

Epoch 256/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7961.0732 - val_loss: 19255.6094
Epoch 257/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7931.3994 - val_loss: 19275.1699
Epoch 258/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7874.9346 - val_loss: 19205.7266
Epoch 259/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7829.9829 - val_loss: 19200.5996
Epoch 260/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7780.3804 - val_loss: 19188.1641
Epoch 261/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7727.1562 - val_loss: 19159.7070
Epoch 262/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7687.8262 - val_loss: 19177.2383
Epoch 263/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7636.7075 - val_loss: 19170.5332
Epoch 264/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7609.0479 - val_loss: 19131.4805
Epoch 265/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7560.3594 - val_loss: 19106.9883
Epoch 266/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss:

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5454.7461 - val_loss: 18774.5273
Epoch 321/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5418.1953 - val_loss: 18782.2461
Epoch 322/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5395.5879 - val_loss: 18762.3945
Epoch 323/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5359.1855 - val_loss: 18743.2656
Epoch 324/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5333.2446 - val_loss: 18728.1895
Epoch 325/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5312.3916 - val_loss: 18757.2188
Epoch 326/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5285.4321 - val_loss: 18710.5176
Epoch 327/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5256.0972 - val_loss: 18707.7539
Epoch 328/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5221.5913 - val_loss: 18719.2539
Epoch 329/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5206.1670 - val_loss: 18676.6035
Epoch 330/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5183.2705 - v

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3959.4673 - val_loss: 18649.4102
Epoch 385/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3950.6665 - val_loss: 18674.4785
Epoch 386/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3931.6050 - val_loss: 18640.2500
Epoch 387/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3910.3296 - val_loss: 18703.5312
Epoch 388/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3907.4331 - val_loss: 18673.9961
Epoch 389/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3880.2546 - val_loss: 18669.2871
Epoch 390/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3854.9744 - val_loss: 18699.8125
Epoch 391/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3835.9358 - val_loss: 18724.4570
Epoch 392/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3830.3901 - val_loss: 18719.3535
Epoch 393/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3804.5854 - val_loss: 18695.4883
Epoch 394/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3774.8096 - v

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2988.8650 - val_loss: 19535.5898
Epoch 449/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2962.0654 - val_loss: 19511.0293
Epoch 450/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2938.3923 - val_loss: 19557.0059
Epoch 451/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2936.1418 - val_loss: 19527.8496
Epoch 452/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2912.1082 - val_loss: 19572.3086
Epoch 453/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2905.4792 - val_loss: 19586.8691
Epoch 454/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2896.8518 - val_loss: 19565.7812
Epoch 455/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2868.6309 - val_loss: 19576.7988
Epoch 456/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2849.5796 - val_loss: 19578.0820
Epoch 457/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2840.2302 - val_loss: 19610.9023
Epoch 458/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2831.7981 - v

In [8]:
if predictions.ndim != 1:
    predictions = predictions.flatten()

# Check if 'y_test' is not 1-dimensional and reshape if necessary
if y_test.ndim != 1:
    y_test = y_test.flatten()
    
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
print(results)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(mae, 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

     Prediction      Actual
0    138.941696  119.793729
1    414.231110  359.563884
2    483.195465  168.934525
3    302.744019  355.505368
4    470.477966  433.151080
..          ...         ...
112  701.003113  487.957347
113  213.901443  216.123044
114  291.354797  425.953734
115  347.237793  436.229287
116  375.809906  388.424770

[117 rows x 2 columns]
Mean Absolute Error (MAE): 85.20444474896809
Mean Squared Error (MSE): 12520.55365534739
Root Mean Squared Error (RMSE): 111.89527986178591
R-squared (R²): 0.778352433409016
Mean Absolute Error: 85.2 degrees.
Accuracy: 64.43 %.


C:\Users\EJ-PC\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [9]:
# build model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # No activation for regression
])

# compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# train model
history = model.fit(X_train, y_train, epochs=1000, batch_size=16, validation_split=0.15)

# evaluate the model
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

#predict
predictions = model.predict(X_test)

C:\Users\EJ-PC\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 219171.3750 - val_loss: 227695.8750
Epoch 2/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 205160.7812 - val_loss: 190076.5469
Epoch 3/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 160322.9531 - val_loss: 129897.7109
Epoch 4/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 105708.7891 - val_loss: 92384.1719
Epoch 5/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 985us/step - loss: 82102.6953 - val_loss: 75307.9844
Epoch 6/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 71974.3906 - val_loss: 66510.8672
Epoch 7/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 66012.9141 - val_loss: 61181.8320
Epoch 8/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 61922.4258 - val_loss: 57331.1367
Epoch 9/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 58679.4258 - val_loss: 54346.6562
Epoch 10/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 55959.0117 - val_loss: 51754.7266
Epoch 11/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/st

56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 26322.6055 - val_loss: 29684.8906
Epoch 65/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 26050.9941 - val_loss: 29572.3848
Epoch 66/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 25797.3125 - val_loss: 29364.5430
Epoch 67/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 25532.0039 - val_loss: 29283.0156
Epoch 68/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 25280.5781 - val_loss: 29100.5664
Epoch 69/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 25021.8574 - val_loss: 29005.5820
Epoch 70/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 24769.4980 - val_loss: 28846.2441
Epoch 71/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 24526.5566 - val_loss: 28729.3027
Epoch 72/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 24274.7305 - val_loss: 28522.2969
Epoch 73/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 24049.1055 - val_loss: 28429.0879
Epoch 74/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 237

56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 13997.6533 - val_loss: 22932.1172
Epoch 128/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 13900.5430 - val_loss: 22855.3164
Epoch 129/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 13774.9062 - val_loss: 22840.0352
Epoch 130/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 13679.9102 - val_loss: 22788.9160
Epoch 131/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1000us/step - loss: 13566.8486 - val_loss: 22746.1016
Epoch 132/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 13468.1357 - val_loss: 22763.0508
Epoch 133/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 13385.6240 - val_loss: 22720.4102
Epoch 134/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 13265.9932 - val_loss: 22559.6445
Epoch 135/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 13154.8662 - val_loss: 22553.5703
Epoch 136/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 13074.6250 - val_loss: 22471.0527
Epoch 137/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/ste

56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 9400.0420 - val_loss: 21480.1211
Epoch 191/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 9391.0879 - val_loss: 21504.2285
Epoch 192/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 9366.6309 - val_loss: 21429.6875
Epoch 193/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 9298.2178 - val_loss: 21434.6133
Epoch 194/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 9270.8066 - val_loss: 21215.2383
Epoch 195/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 9196.5537 - val_loss: 21328.2695
Epoch 196/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 9180.0918 - val_loss: 21365.7695
Epoch 197/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 9152.4775 - val_loss: 21333.0195
Epoch 198/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 9094.4580 - val_loss: 21277.9414
Epoch 199/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 9064.0264 - val_loss: 21001.7031
Epoch 200/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 894

...

Epoch 696/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 589.6692 - val_loss: 22201.9180
Epoch 697/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 597.7244 - val_loss: 22136.1504
Epoch 698/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 985us/step - loss: 561.9055 - val_loss: 22218.0938
Epoch 699/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 993us/step - loss: 578.7108 - val_loss: 22259.3809
Epoch 700/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 574.3434 - val_loss: 22170.1250
Epoch 701/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 571.4042 - val_loss: 22241.4941
Epoch 702/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 590.2876 - val_loss: 22208.6055
Epoch 703/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 596.1397 - val_loss: 22086.7676
Epoch 704/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 528.4512 - val_loss: 22206.7051
Epoch 705/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 573.3508 - val_loss: 22149.1660
Epoch 706/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - 

Epoch 760/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 408.0216 - val_loss: 22530.6152
Epoch 761/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 428.7169 - val_loss: 22662.1738
Epoch 762/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 433.6708 - val_loss: 22535.3848
Epoch 763/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 410.7238 - val_loss: 22636.5410
Epoch 764/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 414.4719 - val_loss: 22613.2012
Epoch 765/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 420.1102 - val_loss: 22452.7598
Epoch 766/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 378.3322 - val_loss: 22723.3379
Epoch 767/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 438.6066 - val_loss: 22560.0625
Epoch 768/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 391.3128 - val_loss: 22558.8516
Epoch 769/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 393.8566 - val_loss: 22598.5430
Epoch 770/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss

Epoch 824/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 288.4133 - val_loss: 22992.3164
Epoch 825/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 295.9347 - val_loss: 22929.5645
Epoch 826/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 306.0264 - val_loss: 22895.5996
Epoch 827/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 294.3617 - val_loss: 22940.0645
Epoch 828/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 980us/step - loss: 292.8270 - val_loss: 23070.1152
Epoch 829/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 968us/step - loss: 291.2582 - val_loss: 23076.3398
Epoch 830/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 297.6478 - val_loss: 23097.1719
Epoch 831/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 298.9791 - val_loss: 23061.1680
Epoch 832/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 289.7296 - val_loss: 23133.5371
Epoch 833/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 293.6887 - val_loss: 23132.7539
Epoch 834/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - 

Epoch 888/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 281.7747 - val_loss: 23925.1523
Epoch 889/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 285.1342 - val_loss: 23867.6211
Epoch 890/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 285.8005 - val_loss: 23799.7559
Epoch 891/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 283.9915 - val_loss: 23802.6250
Epoch 892/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 288.9221 - val_loss: 23730.4102
Epoch 893/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 309.1270 - val_loss: 23709.6953
Epoch 894/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 313.3381 - val_loss: 23603.4609
Epoch 895/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 291.2422 - val_loss: 23598.5742
Epoch 896/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 293.9559 - val_loss: 23606.6875
Epoch 897/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 315.8491 - val_loss: 23582.5039
Epoch 898/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss

56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 843.2055 - val_loss: 24783.6367
Epoch 953/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 957.7520 - val_loss: 25130.5586
Epoch 954/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 996us/step - loss: 1127.7814 - val_loss: 25563.6836
Epoch 955/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1318.7747 - val_loss: 25824.1289
Epoch 956/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1474.1520 - val_loss: 25909.5059
Epoch 957/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1586.3549 - val_loss: 25630.6172
Epoch 958/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1543.3287 - val_loss: 25231.6973
Epoch 959/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1423.1993 - val_loss: 24834.9902
Epoch 960/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1290.4169 - val_loss: 24385.3477
Epoch 961/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1152.1501 - val_loss: 24014.0898
Epoch 962/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 112

In [10]:
if predictions.ndim != 1:
    predictions = predictions.flatten()

# Check if 'y_test' is not 1-dimensional and reshape if necessary
if y_test.ndim != 1:
    y_test = y_test.flatten()
    
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
print(results)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(mae, 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

     Prediction      Actual
0    187.172699  119.793729
1    307.622864  359.563884
2    284.080780  168.934525
3    218.337906  355.505368
4    525.490845  433.151080
..          ...         ...
112  662.828552  487.957347
113  140.918762  216.123044
114  349.045776  425.953734
115  332.769836  436.229287
116  368.685822  388.424770

[117 rows x 2 columns]
Mean Absolute Error (MAE): 95.17005872613571
Mean Squared Error (MSE): 15931.122612690642
Root Mean Squared Error (RMSE): 126.2185509847528
R-squared (R²): 0.7179761648433665
Mean Absolute Error: 95.17 degrees.
Accuracy: 63.21 %.


C:\Users\EJ-PC\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [11]:
# build model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)  # No activation for regression
])

# compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# train model
history = model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_split=0.15)

# evaluate the model
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

#predict
predictions = model.predict(X_test)

Epoch 1/1000


C:\Users\EJ-PC\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 220447.0000 - val_loss: 231050.6562
Epoch 2/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 212071.6875 - val_loss: 212556.2500
Epoch 3/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 189415.0781 - val_loss: 174855.7500
Epoch 4/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 148137.7812 - val_loss: 126169.6250
Epoch 5/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 102796.0625 - val_loss: 87610.9062
Epoch 6/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 78299.5312 - val_loss: 71959.6406
Epoch 7/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 70619.2891 - val_loss: 64818.2617
Epoch 8/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 65541.9844 - val_loss: 60369.3242
Epoch 9/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 61808.5000 - val_loss: 56900.9766
Epoch 10/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 58739.4922 - val_loss: 54028.0195
Epoch 11/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 56

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 22383.1992 - val_loss: 27284.1680
Epoch 65/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 22084.1680 - val_loss: 27131.1973
Epoch 66/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 21789.9004 - val_loss: 27025.9121
Epoch 67/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 21510.6055 - val_loss: 26872.9434
Epoch 68/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 21233.8223 - val_loss: 26737.6094
Epoch 69/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 20960.0352 - val_loss: 26626.6758
Epoch 70/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 20686.5918 - val_loss: 26495.8633
Epoch 71/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 20419.4238 - val_loss: 26425.4160
Epoch 72/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 20159.5840 - val_loss: 26299.9258
Epoch 73/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 19896.4141 - val_loss: 26181.9121
Epoch 74/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 196

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 10975.9932 - val_loss: 22093.6406
Epoch 128/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 10885.3203 - val_loss: 22105.4688
Epoch 129/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 10788.8730 - val_loss: 22113.9180
Epoch 130/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 10720.5215 - val_loss: 22042.5078
Epoch 131/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 10636.7295 - val_loss: 22094.6055
Epoch 132/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 10539.9717 - val_loss: 22068.5117
Epoch 133/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 10487.7148 - val_loss: 22056.2773
Epoch 134/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 10412.0059 - val_loss: 22028.3418
Epoch 135/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 10321.1104 - val_loss: 22016.9160
Epoch 136/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 10246.1592 - val_loss: 21929.2617
Epoch 137/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step -

...

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 487.8193 - val_loss: 22748.1191
Epoch 699/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 495.2883 - val_loss: 22761.3945
Epoch 700/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 475.5935 - val_loss: 22799.1777
Epoch 701/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 477.3435 - val_loss: 22944.6836
Epoch 702/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 530.0067 - val_loss: 22863.0137
Epoch 703/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 526.1847 - val_loss: 22882.8438
Epoch 704/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 543.3317 - val_loss: 22880.4316
Epoch 705/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 559.9065 - val_loss: 22686.7500
Epoch 706/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 524.8406 - val_loss: 22645.3594
Epoch 707/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 526.6613 - val_loss: 22546.4609
Epoch 708/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 513.4606 - va

Epoch 762/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 350.5151 - val_loss: 21282.9922
Epoch 763/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 322.2932 - val_loss: 21295.9824
Epoch 764/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 308.8560 - val_loss: 21287.8594
Epoch 765/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 287.6424 - val_loss: 21288.0137
Epoch 766/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 277.1383 - val_loss: 21284.9492
Epoch 767/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 258.2831 - val_loss: 21289.7812
Epoch 768/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 252.5148 - val_loss: 21264.4062
Epoch 769/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 234.3461 - val_loss: 21288.3594
Epoch 770/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 231.8879 - val_loss: 21279.5723
Epoch 771/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 217.0593 - val_loss: 21285.8750
Epoch 772/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 160.3031 - val_loss: 21648.2520
Epoch 827/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 121.3943 - val_loss: 21648.9414
Epoch 828/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 176.0002 - val_loss: 21580.2285
Epoch 829/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 131.9172 - val_loss: 21679.2852
Epoch 830/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 186.4722 - val_loss: 21686.8555
Epoch 831/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 154.2363 - val_loss: 21853.0371
Epoch 832/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 270.7554 - val_loss: 21772.5098
Epoch 833/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 247.1258 - val_loss: 21706.2695
Epoch 834/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 253.2768 - val_loss: 21708.1055
Epoch 835/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 215.0452 - val_loss: 21708.6523
Epoch 836/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 344.3154 - va

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 384.3758 - val_loss: 27327.3398
Epoch 891/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 402.6305 - val_loss: 27337.8652
Epoch 892/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 396.9621 - val_loss: 27483.2930
Epoch 893/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 417.8601 - val_loss: 27519.2559
Epoch 894/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 406.7850 - val_loss: 27789.7812
Epoch 895/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 439.8773 - val_loss: 27852.0215
Epoch 896/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 438.9580 - val_loss: 28104.0039
Epoch 897/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 467.5988 - val_loss: 28153.4746
Epoch 898/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 462.9789 - val_loss: 28288.8809
Epoch 899/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 486.8615 - val_loss: 28432.7246
Epoch 900/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 502.4472 - va

Epoch 954/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4197.4604 - val_loss: 21099.5234
Epoch 955/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4456.6763 - val_loss: 21257.8340
Epoch 956/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4432.0117 - val_loss: 21017.1914
Epoch 957/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3979.7378 - val_loss: 20781.6309
Epoch 958/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3370.4216 - val_loss: 20612.8535
Epoch 959/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2800.6462 - val_loss: 20550.3926
Epoch 960/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2315.6272 - val_loss: 20606.8477
Epoch 961/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1957.7827 - val_loss: 20683.8457
Epoch 962/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1712.0967 - val_loss: 20767.3398
Epoch 963/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1527.2234 - val_loss: 20811.8320
Epoch 964/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/s

In [12]:
if predictions.ndim != 1:
    predictions = predictions.flatten()

# Check if 'y_test' is not 1-dimensional and reshape if necessary
if y_test.ndim != 1:
    y_test = y_test.flatten()
    
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
print(results)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(mae, 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

     Prediction      Actual
0    175.423965  119.793729
1    354.450256  359.563884
2    475.521545  168.934525
3    277.768921  355.505368
4    566.184265  433.151080
..          ...         ...
112  560.387512  487.957347
113  209.255569  216.123044
114  340.621338  425.953734
115  303.316559  436.229287
116  272.354309  388.424770

[117 rows x 2 columns]
Mean Absolute Error (MAE): 89.2732567401494
Mean Squared Error (MSE): 15315.861349592147
Root Mean Squared Error (RMSE): 123.75726786573848
R-squared (R²): 0.7288679485086387
Mean Absolute Error: 89.27 degrees.
Accuracy: 65.12 %.


C:\Users\EJ-PC\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [13]:
# build model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)  # No activation for regression
])

# compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# train model
history = model.fit(X_train, y_train, epochs=500, batch_size=32, validation_split=0.15)

# evaluate the model
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

#predict
predictions = model.predict(X_test)

Epoch 1/500


C:\Users\EJ-PC\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 220259.5781 - val_loss: 231274.9531
Epoch 2/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 212214.5469 - val_loss: 212335.2031
Epoch 3/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 188779.8906 - val_loss: 171150.5625
Epoch 4/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 144621.5938 - val_loss: 121490.4219
Epoch 5/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 100419.7422 - val_loss: 88498.1641
Epoch 6/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 80375.6797 - val_loss: 74665.5547
Epoch 7/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 73049.2266 - val_loss: 67675.6641
Epoch 8/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 67887.8984 - val_loss: 63118.5391
Epoch 9/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 64083.5156 - val_loss: 59439.9688
Epoch 10/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 60907.5039 - val_loss: 56322.9180
Epoch 11/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 58084.2305 -

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 23667.6699 - val_loss: 28073.0215
Epoch 66/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 23391.7266 - val_loss: 27942.7441
Epoch 67/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 23133.1289 - val_loss: 27814.8242
Epoch 68/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 22871.9395 - val_loss: 27693.7031
Epoch 69/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 22630.6523 - val_loss: 27577.0039
Epoch 70/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 22386.6621 - val_loss: 27488.9258
Epoch 71/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 22163.6621 - val_loss: 27358.3691
Epoch 72/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 21925.1680 - val_loss: 27258.0879
Epoch 73/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 21706.9219 - val_loss: 27126.5977
Epoch 74/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 21478.6621 - val_loss: 27036.3809
Epoch 75/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 21274.6367 - 

Epoch 129/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 11918.9385 - val_loss: 22494.7441
Epoch 130/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 11807.0459 - val_loss: 22505.7285
Epoch 131/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 11723.6836 - val_loss: 22438.8672
Epoch 132/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 11614.9561 - val_loss: 22322.7285
Epoch 133/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 11516.0742 - val_loss: 22309.1992
Epoch 134/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 11404.3633 - val_loss: 22366.5684
Epoch 135/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 11355.7842 - val_loss: 22178.6816
Epoch 136/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 11224.1211 - val_loss: 22197.8086
Epoch 137/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 11154.7822 - val_loss: 22100.5273
Epoch 138/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 11050.5391 - val_loss: 22113.4570
Epoch 139/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/st

Epoch 193/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7903.6870 - val_loss: 20321.1152
Epoch 194/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7827.4189 - val_loss: 20316.1484
Epoch 195/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7791.3462 - val_loss: 20339.7305
Epoch 196/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7762.8208 - val_loss: 20270.1699
Epoch 197/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7710.4590 - val_loss: 20251.7695
Epoch 198/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7660.8530 - val_loss: 20261.1094
Epoch 199/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7636.0576 - val_loss: 20222.4941
Epoch 200/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7590.3628 - val_loss: 20205.4238
Epoch 201/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7553.4014 - val_loss: 20191.1406
Epoch 202/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7502.2749 - val_loss: 20186.2773
Epoch 203/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss:

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5495.9951 - val_loss: 18736.3867
Epoch 258/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5451.7852 - val_loss: 18693.5957
Epoch 259/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5402.4565 - val_loss: 18733.3906
Epoch 260/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5398.9058 - val_loss: 18644.8789
Epoch 261/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5339.6675 - val_loss: 18663.1387
Epoch 262/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5322.0986 - val_loss: 18618.6992
Epoch 263/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5283.2266 - val_loss: 18607.4727
Epoch 264/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5250.9648 - val_loss: 18573.9648
Epoch 265/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5214.3047 - val_loss: 18552.6035
Epoch 266/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5183.7290 - val_loss: 18491.6895
Epoch 267/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5141.7515 - v

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3693.2861 - val_loss: 17995.4336
Epoch 322/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3666.2478 - val_loss: 17932.1641
Epoch 323/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3642.8723 - val_loss: 17955.4492
Epoch 324/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3613.9419 - val_loss: 17896.2246
Epoch 325/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3574.5564 - val_loss: 17960.2012
Epoch 326/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3588.7246 - val_loss: 17877.7188
Epoch 327/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3527.2334 - val_loss: 17979.6758
Epoch 328/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3550.7668 - val_loss: 17881.5664
Epoch 329/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3482.7964 - val_loss: 17997.4336
Epoch 330/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3507.4841 - val_loss: 17880.0469
Epoch 331/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3441.5159 - v

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2649.5667 - val_loss: 18819.1152
Epoch 386/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2621.8096 - val_loss: 18856.7773
Epoch 387/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2649.6943 - val_loss: 18737.9141
Epoch 388/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2560.5447 - val_loss: 18958.4395
Epoch 389/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2616.9241 - val_loss: 18754.7656
Epoch 390/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2544.0142 - val_loss: 18684.3066
Epoch 391/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2474.3792 - val_loss: 18759.3945
Epoch 392/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2497.6917 - val_loss: 18773.6719
Epoch 393/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2482.7053 - val_loss: 18754.2832
Epoch 394/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2449.6938 - val_loss: 18727.8750
Epoch 395/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2424.6582 - v

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1855.7499 - val_loss: 18444.2754
Epoch 450/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1546.8335 - val_loss: 18443.6348
Epoch 451/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1582.0737 - val_loss: 18372.6875
Epoch 452/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1452.3641 - val_loss: 18374.0117
Epoch 453/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1463.2750 - val_loss: 18356.8730
Epoch 454/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1409.3352 - val_loss: 18354.5430
Epoch 455/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1404.8718 - val_loss: 18373.2793
Epoch 456/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1379.5651 - val_loss: 18355.4082
Epoch 457/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1374.8378 - val_loss: 18366.9316
Epoch 458/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1357.8124 - val_loss: 18358.7305
Epoch 459/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1348.6030 - v

In [14]:
if predictions.ndim != 1:
    predictions = predictions.flatten()

# Check if 'y_test' is not 1-dimensional and reshape if necessary
if y_test.ndim != 1:
    y_test = y_test.flatten()
    
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
print(results)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(mae, 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

     Prediction      Actual
0    246.357803  119.793729
1    311.812927  359.563884
2    398.296875  168.934525
3    283.498016  355.505368
4    552.793335  433.151080
..          ...         ...
112  685.476074  487.957347
113  156.715897  216.123044
114  310.274231  425.953734
115  382.359863  436.229287
116  351.984406  388.424770

[117 rows x 2 columns]
Mean Absolute Error (MAE): 87.94150431412521
Mean Squared Error (MSE): 13054.111481798507
Root Mean Squared Error (RMSE): 114.25459063774422
R-squared (R²): 0.7689070209197724
Mean Absolute Error: 87.94 degrees.
Accuracy: 66.29 %.


C:\Users\EJ-PC\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [15]:
# build model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)  # No activation for regression
])

# compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# train model
history = model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_split=0.15)

# evaluate the model
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

#predict
predictions = model.predict(X_test)

Epoch 1/1000


C:\Users\EJ-PC\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 219011.0000 - val_loss: 224943.5469
Epoch 2/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 202570.7344 - val_loss: 186859.0781
Epoch 3/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 158306.9688 - val_loss: 126818.7891
Epoch 4/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 101970.2656 - val_loss: 82596.2031
Epoch 5/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 76742.1172 - val_loss: 67969.8594
Epoch 6/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 68682.1797 - val_loss: 61394.8945
Epoch 7/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 62991.6797 - val_loss: 56884.9688
Epoch 8/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 58917.3516 - val_loss: 53166.3633
Epoch 9/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 55457.3789 - val_loss: 50363.8281
Epoch 10/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 52565.9258 - val_loss: 47632.6016
Epoch 11/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4996

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 21396.8965 - val_loss: 26429.4688
Epoch 65/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 21055.6094 - val_loss: 26295.4844
Epoch 66/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 20706.7129 - val_loss: 26162.0098
Epoch 67/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 20377.0098 - val_loss: 26031.0117
Epoch 68/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 20048.7637 - val_loss: 25898.0215
Epoch 69/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 19743.4121 - val_loss: 25750.5586
Epoch 70/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 19441.1875 - val_loss: 25613.6035
Epoch 71/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 19147.7070 - val_loss: 25492.5039
Epoch 72/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 18850.9512 - val_loss: 25331.9414
Epoch 73/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 18579.6250 - val_loss: 25179.1836
Epoch 74/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 182

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9132.3682 - val_loss: 20180.9785
Epoch 128/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9062.7002 - val_loss: 20143.8691
Epoch 129/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8979.9395 - val_loss: 20177.4668
Epoch 130/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8911.5098 - val_loss: 20124.8223
Epoch 131/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8841.4424 - val_loss: 20188.7070
Epoch 132/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8773.2139 - val_loss: 20163.8535
Epoch 133/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8693.8369 - val_loss: 20125.2676
Epoch 134/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8625.5000 - val_loss: 20085.6133
Epoch 135/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8549.4326 - val_loss: 20080.2539
Epoch 136/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8489.4482 - val_loss: 20104.4492
Epoch 137/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 841

...

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 234.5711 - val_loss: 20690.4883
Epoch 698/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 212.0622 - val_loss: 20733.4492
Epoch 699/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 218.9849 - val_loss: 20714.9414
Epoch 700/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 209.9099 - val_loss: 20749.5801
Epoch 701/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 213.4849 - val_loss: 20790.0879
Epoch 702/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 207.9541 - val_loss: 20792.0977
Epoch 703/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 212.1785 - val_loss: 20780.3984
Epoch 704/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 208.2188 - val_loss: 20772.9746
Epoch 705/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 213.4984 - val_loss: 20732.0723
Epoch 706/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 205.4709 - val_loss: 20711.6387
Epoch 707/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 209.1525 - va

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 297.2753 - val_loss: 21114.8926
Epoch 762/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 277.5773 - val_loss: 21007.6504
Epoch 763/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 286.2387 - val_loss: 20817.2148
Epoch 764/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 284.5868 - val_loss: 20595.3340
Epoch 765/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 295.4937 - val_loss: 20455.4141
Epoch 766/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 313.8191 - val_loss: 20292.6914
Epoch 767/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 316.5990 - val_loss: 20278.9941
Epoch 768/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 317.2026 - val_loss: 20343.3086
Epoch 769/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 327.1436 - val_loss: 20628.5547
Epoch 770/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 333.7892 - val_loss: 20922.9062
Epoch 771/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 341.2216 - va

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 416.6551 - val_loss: 20456.7910
Epoch 826/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 417.8517 - val_loss: 20888.5508
Epoch 827/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 472.8457 - val_loss: 21234.0273
Epoch 828/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 567.5089 - val_loss: 21456.4316
Epoch 829/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 665.1995 - val_loss: 21378.4277
Epoch 830/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 659.4396 - val_loss: 20910.2285
Epoch 831/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 507.1790 - val_loss: 20645.9863
Epoch 832/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 480.7108 - val_loss: 20369.1641
Epoch 833/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 569.3898 - val_loss: 20405.2012
Epoch 834/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 828.0313 - val_loss: 19977.5977
Epoch 835/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 915.1545 - va

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 620.3031 - val_loss: 21433.5430
Epoch 890/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 645.2133 - val_loss: 21561.3848
Epoch 891/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 677.2005 - val_loss: 21629.6387
Epoch 892/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 688.7254 - val_loss: 21744.2598
Epoch 893/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 705.2748 - val_loss: 21812.6973
Epoch 894/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 728.9775 - val_loss: 21848.8711
Epoch 895/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 758.5365 - val_loss: 21749.4805
Epoch 896/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 780.9725 - val_loss: 21692.4492
Epoch 897/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 824.8874 - val_loss: 21561.4062
Epoch 898/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 858.1889 - val_loss: 21458.6191
Epoch 899/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 925.3257 - va

Epoch 953/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 207.4095 - val_loss: 22060.3281
Epoch 954/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 180.4195 - val_loss: 22204.2930
Epoch 955/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 197.9123 - val_loss: 22176.3379
Epoch 956/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 173.1739 - val_loss: 22318.4102
Epoch 957/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 192.5392 - val_loss: 22283.1113
Epoch 958/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 164.5633 - val_loss: 22489.8867
Epoch 959/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 197.7109 - val_loss: 22366.0039
Epoch 960/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 152.8385 - val_loss: 22625.0938
Epoch 961/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 211.0538 - val_loss: 22410.3340
Epoch 962/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 147.2571 - val_loss: 22913.4844
Epoch 963/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss

In [16]:
if predictions.ndim != 1:
    predictions = predictions.flatten()

# Check if 'y_test' is not 1-dimensional and reshape if necessary
if y_test.ndim != 1:
    y_test = y_test.flatten()
    
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
print(results)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(mae, 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

     Prediction      Actual
0    179.137482  119.793729
1    224.878922  359.563884
2    343.411804  168.934525
3    371.414062  355.505368
4    544.538208  433.151080
..          ...         ...
112  625.511108  487.957347
113  285.076172  216.123044
114  319.158203  425.953734
115  333.886261  436.229287
116  493.315887  388.424770

[117 rows x 2 columns]
Mean Absolute Error (MAE): 97.46397395650781
Mean Squared Error (MSE): 14712.532876732157
Root Mean Squared Error (RMSE): 121.29523023075622
R-squared (R²): 0.7395484896049469
Mean Absolute Error: 97.46 degrees.
Accuracy: 64.31 %.


C:\Users\EJ-PC\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [17]:
# build model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)  # No activation for regression
])

# compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# train model
history = model.fit(X_train, y_train, epochs=500, batch_size=32, validation_split=0.15)

# evaluate the model
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

#predict
predictions = model.predict(X_test)

Epoch 1/500


C:\Users\EJ-PC\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 220170.8125 - val_loss: 229767.7500
Epoch 2/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 209645.1406 - val_loss: 204256.4688
Epoch 3/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 178148.3750 - val_loss: 155504.4062
Epoch 4/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 125870.1406 - val_loss: 100997.2969
Epoch 5/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 85437.5547 - val_loss: 75883.1875
Epoch 6/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 73235.9453 - val_loss: 65378.8828
Epoch 7/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 65979.1875 - val_loss: 59729.6953
Epoch 8/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 61304.7109 - val_loss: 55902.0703
Epoch 9/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 57786.0625 - val_loss: 52782.1797
Epoch 10/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 54769.2148 - val_loss: 50185.7305
Epoch 11/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 52158.3555 - 

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 20667.7324 - val_loss: 26456.4414
Epoch 66/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 20342.3164 - val_loss: 26318.9277
Epoch 67/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 19999.2070 - val_loss: 26111.7969
Epoch 68/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 19666.1875 - val_loss: 25925.7344
Epoch 69/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 19308.2246 - val_loss: 25739.8633
Epoch 70/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 18999.9395 - val_loss: 25570.5898
Epoch 71/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 18659.0352 - val_loss: 25409.0527
Epoch 72/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 18343.8438 - val_loss: 25187.3555
Epoch 73/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 18002.0176 - val_loss: 25016.9336
Epoch 74/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 17681.2930 - val_loss: 24780.6914
Epoch 75/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 17347.2988 - 

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7893.1865 - val_loss: 19568.1992
Epoch 130/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7813.3931 - val_loss: 19457.7500
Epoch 131/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7726.3628 - val_loss: 19422.7676
Epoch 132/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7642.6128 - val_loss: 19410.1680
Epoch 133/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7570.5718 - val_loss: 19369.4336
Epoch 134/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7483.0869 - val_loss: 19306.9297
Epoch 135/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7404.0991 - val_loss: 19277.2754
Epoch 136/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7328.1377 - val_loss: 19251.6152
Epoch 137/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7256.8794 - val_loss: 19215.9238
Epoch 138/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7180.2139 - val_loss: 19160.8457
Epoch 139/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7088.9907 - v

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4085.1895 - val_loss: 17831.2773
Epoch 194/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3845.2937 - val_loss: 18503.3320
Epoch 195/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4117.5820 - val_loss: 17900.5527
Epoch 196/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3777.1724 - val_loss: 18247.3691
Epoch 197/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3865.1050 - val_loss: 18067.1855
Epoch 198/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3744.2673 - val_loss: 18032.5664
Epoch 199/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3693.2297 - val_loss: 18118.1992
Epoch 200/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3697.5439 - val_loss: 18198.9355
Epoch 201/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3645.5593 - val_loss: 18216.3125
Epoch 202/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3620.6658 - val_loss: 18217.9375
Epoch 203/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3584.7188 - v

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1884.9850 - val_loss: 18379.2148
Epoch 258/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1870.2896 - val_loss: 18368.1348
Epoch 259/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1850.5645 - val_loss: 18363.0293
Epoch 260/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1816.0775 - val_loss: 18341.7148
Epoch 261/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1795.4980 - val_loss: 18370.6504
Epoch 262/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1772.9427 - val_loss: 18401.2461
Epoch 263/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1756.3588 - val_loss: 18361.3984
Epoch 264/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1724.7349 - val_loss: 18423.6895
Epoch 265/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1715.1818 - val_loss: 18386.1816
Epoch 266/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1679.0002 - val_loss: 18368.1094
Epoch 267/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1656.7401 - v

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 881.8480 - val_loss: 18564.7988
Epoch 322/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 847.2205 - val_loss: 18678.1836
Epoch 323/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 857.4214 - val_loss: 18583.3906
Epoch 324/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 821.8679 - val_loss: 18663.2930
Epoch 325/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 834.6820 - val_loss: 18615.9609
Epoch 326/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 804.3965 - val_loss: 18696.0352
Epoch 327/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 814.8366 - val_loss: 18660.0293
Epoch 328/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 783.0681 - val_loss: 18717.7949
Epoch 329/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 797.9186 - val_loss: 18672.2715
Epoch 330/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 767.7396 - val_loss: 18761.1777
Epoch 331/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 778.7100 - val_loss: 18

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1140.6273 - val_loss: 20560.7031
Epoch 386/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1255.5099 - val_loss: 20497.0938
Epoch 387/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1231.2432 - val_loss: 20383.4531
Epoch 388/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 912.6589 - val_loss: 20600.1895
Epoch 389/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 744.2166 - val_loss: 20716.3164
Epoch 390/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 683.3836 - val_loss: 20724.7500
Epoch 391/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 680.9058 - val_loss: 20702.9863
Epoch 392/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 656.3774 - val_loss: 20618.9473
Epoch 393/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 631.9813 - val_loss: 20675.9219
Epoch 394/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 682.5273 - val_loss: 20856.7871
Epoch 395/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 674.1527 - val_loss:

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 671.6751 - val_loss: 22101.4004
Epoch 450/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 659.3979 - val_loss: 22136.2168
Epoch 451/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 660.8895 - val_loss: 22127.3789
Epoch 452/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 676.3890 - val_loss: 22090.4238
Epoch 453/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 672.8056 - val_loss: 22056.4570
Epoch 454/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 691.0954 - val_loss: 21961.8633
Epoch 455/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 721.7686 - val_loss: 21700.8535
Epoch 456/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 720.6648 - val_loss: 21452.9590
Epoch 457/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 738.8118 - val_loss: 21153.8477
Epoch 458/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 755.0929 - val_loss: 20923.5625
Epoch 459/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 796.9941 - val_loss: 20

In [18]:
if predictions.ndim != 1:
    predictions = predictions.flatten()

# Check if 'y_test' is not 1-dimensional and reshape if necessary
if y_test.ndim != 1:
    y_test = y_test.flatten()
    
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
print(results)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(mae, 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

     Prediction      Actual
0    185.867691  119.793729
1    360.004608  359.563884
2    367.226990  168.934525
3    225.806808  355.505368
4    587.918701  433.151080
..          ...         ...
112  693.785706  487.957347
113  237.221298  216.123044
114  414.616943  425.953734
115  462.419312  436.229287
116  261.064392  388.424770

[117 rows x 2 columns]
Mean Absolute Error (MAE): 88.16866251456331
Mean Squared Error (MSE): 13489.170142579398
Root Mean Squared Error (RMSE): 116.14288674981088
R-squared (R²): 0.7612053093069451
Mean Absolute Error: 88.17 degrees.
Accuracy: 66.15 %.


C:\Users\EJ-PC\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [19]:
# build model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)  # No activation for regression
])

# compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# train model
history = model.fit(X_train, y_train, epochs=500, batch_size=32, validation_split=0.15)

# evaluate the model
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

#predict
predictions = model.predict(X_test)

Epoch 1/500


C:\Users\EJ-PC\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 219751.6562 - val_loss: 227854.7812
Epoch 2/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 206892.3906 - val_loss: 196361.6094
Epoch 3/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 168843.4219 - val_loss: 139615.0781
Epoch 4/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 111546.5547 - val_loss: 87566.2656
Epoch 5/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 77947.8672 - val_loss: 69424.5312
Epoch 6/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 69545.0156 - val_loss: 62492.0391
Epoch 7/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 63955.4922 - val_loss: 57980.7305
Epoch 8/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 59916.5352 - val_loss: 54451.9492
Epoch 9/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 56618.0859 - val_loss: 51386.1016
Epoch 10/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 53745.6992 - val_loss: 48776.0508
Epoch 11/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 51181.7969 - v

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 18288.8105 - val_loss: 25269.3203
Epoch 66/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 17994.0098 - val_loss: 25095.3223
Epoch 67/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 17705.8457 - val_loss: 24963.5195
Epoch 68/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 17441.5840 - val_loss: 24845.8340
Epoch 69/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 17172.5371 - val_loss: 24698.0703
Epoch 70/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 16915.7578 - val_loss: 24590.3184
Epoch 71/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 16661.0293 - val_loss: 24424.8633
Epoch 72/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 16406.1230 - val_loss: 24257.2773
Epoch 73/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 16169.9932 - val_loss: 24280.9492
Epoch 74/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 15947.1191 - val_loss: 24152.3613
Epoch 75/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 15719.0986 - 

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8387.2373 - val_loss: 20343.6934
Epoch 130/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8307.1191 - val_loss: 20317.4004
Epoch 131/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8243.3447 - val_loss: 20295.6855
Epoch 132/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8154.6118 - val_loss: 20298.0762
Epoch 133/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8103.1377 - val_loss: 20246.3945
Epoch 134/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8021.3501 - val_loss: 20227.8359
Epoch 135/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7958.4004 - val_loss: 20155.6836
Epoch 136/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7873.7334 - val_loss: 20166.0605
Epoch 137/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7830.2871 - val_loss: 20134.6484
Epoch 138/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7751.6387 - val_loss: 20126.8496
Epoch 139/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7691.4189 - v

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4836.8950 - val_loss: 19146.0371
Epoch 194/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4773.1685 - val_loss: 19214.6211
Epoch 195/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4768.1626 - val_loss: 19168.6543
Epoch 196/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4699.7754 - val_loss: 19188.1328
Epoch 197/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4678.9351 - val_loss: 19168.1309
Epoch 198/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4608.9810 - val_loss: 19140.3438
Epoch 199/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4583.3218 - val_loss: 19134.2051
Epoch 200/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4547.2666 - val_loss: 19156.1582
Epoch 201/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4505.7466 - val_loss: 19067.1406
Epoch 202/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4456.8833 - val_loss: 19060.4082
Epoch 203/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4391.1167 - v

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2765.3459 - val_loss: 19140.0781
Epoch 258/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2759.2600 - val_loss: 19140.1035
Epoch 259/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2715.2219 - val_loss: 19143.9766
Epoch 260/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2702.0771 - val_loss: 19153.1152
Epoch 261/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2665.7534 - val_loss: 19144.7695
Epoch 262/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2655.7974 - val_loss: 19096.3945
Epoch 263/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2598.4309 - val_loss: 19239.7637
Epoch 264/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2684.8943 - val_loss: 19123.8008
Epoch 265/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2580.0286 - val_loss: 19318.7812
Epoch 266/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2654.8423 - val_loss: 19083.8750
Epoch 267/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2521.7021 - v

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1769.3574 - val_loss: 19530.3945
Epoch 322/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1561.8839 - val_loss: 20023.9102
Epoch 323/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1793.3944 - val_loss: 19456.5840
Epoch 324/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1475.4843 - val_loss: 19691.3789
Epoch 325/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1574.1903 - val_loss: 19404.5723
Epoch 326/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1407.8615 - val_loss: 19588.8301
Epoch 327/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1474.0262 - val_loss: 19455.8984
Epoch 328/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1387.7114 - val_loss: 19521.5977
Epoch 329/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1413.7932 - val_loss: 19457.7500
Epoch 330/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1359.9906 - val_loss: 19545.3027
Epoch 331/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1366.2987 - v

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 749.1799 - val_loss: 20661.2402
Epoch 386/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 891.3635 - val_loss: 20350.9258
Epoch 387/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 785.3146 - val_loss: 20952.5371
Epoch 388/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1059.4468 - val_loss: 20784.8379
Epoch 389/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1020.5204 - val_loss: 21915.9082
Epoch 390/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1392.4181 - val_loss: 20790.6543
Epoch 391/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 884.8489 - val_loss: 20912.1562
Epoch 392/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 852.9406 - val_loss: 20686.4316
Epoch 393/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 742.5247 - val_loss: 20659.1660
Epoch 394/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 722.0307 - val_loss: 20688.6719
Epoch 395/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 728.6074 - val_loss:

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 399.0699 - val_loss: 21535.9844
Epoch 450/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 396.6844 - val_loss: 21548.4160
Epoch 451/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 392.7489 - val_loss: 21566.0098
Epoch 452/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 389.4239 - val_loss: 21575.8027
Epoch 453/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 385.3544 - val_loss: 21596.4336
Epoch 454/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 382.1540 - val_loss: 21639.3789
Epoch 455/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 378.3275 - val_loss: 21639.2559
Epoch 456/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 372.8227 - val_loss: 21667.1055
Epoch 457/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 368.4178 - val_loss: 21634.2520
Epoch 458/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 363.9507 - val_loss: 21680.1816
Epoch 459/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 356.3523 - val_loss: 21

In [20]:
if predictions.ndim != 1:
    predictions = predictions.flatten()

# Check if 'y_test' is not 1-dimensional and reshape if necessary
if y_test.ndim != 1:
    y_test = y_test.flatten()
    
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
print(results)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(mae, 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

     Prediction      Actual
0    195.931473  119.793729
1    223.701904  359.563884
2    300.434875  168.934525
3    295.272552  355.505368
4    417.252625  433.151080
..          ...         ...
112  571.627197  487.957347
113  147.471756  216.123044
114  225.490677  425.953734
115  317.387512  436.229287
116  224.799362  388.424770

[117 rows x 2 columns]
Mean Absolute Error (MAE): 102.71244250894108
Mean Squared Error (MSE): 17166.09855836778
Root Mean Squared Error (RMSE): 131.01945870124703
R-squared (R²): 0.6961137599775217
Mean Absolute Error: 102.71 degrees.
Accuracy: 65.26 %.


C:\Users\EJ-PC\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [21]:
# build model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)  # No activation for regression
])

# compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# train model
history = model.fit(X_train, y_train, epochs=500, batch_size=32, validation_split=0.15)

# evaluate the model
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

#predict
predictions = model.predict(X_test)

Epoch 1/500


C:\Users\EJ-PC\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 218159.0625 - val_loss: 206813.2031
Epoch 2/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 162698.6406 - val_loss: 89814.1016
Epoch 3/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 76648.3125 - val_loss: 58920.4492
Epoch 4/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 58649.3125 - val_loss: 52113.3203
Epoch 5/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 52286.8203 - val_loss: 46117.6250
Epoch 6/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 47311.9023 - val_loss: 42366.7969
Epoch 7/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 43708.7500 - val_loss: 40277.1992
Epoch 8/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 41546.5234 - val_loss: 38044.8672
Epoch 9/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 39590.3594 - val_loss: 36923.5977
Epoch 10/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 38283.2148 - val_loss: 35644.9141
Epoch 11/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 37006.6055 - val_l

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4944.7339 - val_loss: 21689.5879
Epoch 66/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4811.2646 - val_loss: 21675.7383
Epoch 67/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4675.6230 - val_loss: 21909.8594
Epoch 68/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4592.0005 - val_loss: 21891.1699
Epoch 69/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4467.9668 - val_loss: 22107.8496
Epoch 70/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4455.7334 - val_loss: 22098.3887
Epoch 71/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 4314.7261 - val_loss: 22237.3652
Epoch 72/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4214.2769 - val_loss: 21951.6055
Epoch 73/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3974.4727 - val_loss: 21948.1680
Epoch 74/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3783.9697 - val_loss: 21859.0469
Epoch 75/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3635.2976 - val_loss: 2

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5890.3301 - val_loss: 26190.4902
Epoch 130/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4080.4355 - val_loss: 23003.9785
Epoch 131/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2330.1592 - val_loss: 22838.2969
Epoch 132/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1825.2173 - val_loss: 23239.4902
Epoch 133/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1680.1305 - val_loss: 23151.6621
Epoch 134/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1484.9873 - val_loss: 23053.5703
Epoch 135/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1390.3809 - val_loss: 22877.1172
Epoch 136/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1240.4395 - val_loss: 22724.0684
Epoch 137/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1170.4888 - val_loss: 22505.1875
Epoch 138/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1070.3114 - val_loss: 22348.1777
Epoch 139/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1057.1421 - v

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 421.8452 - val_loss: 21826.0195
Epoch 194/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 412.0041 - val_loss: 21803.9023
Epoch 195/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 397.5662 - val_loss: 21798.7637
Epoch 196/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 382.0531 - val_loss: 21815.1406
Epoch 197/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 368.6324 - val_loss: 21768.6309
Epoch 198/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 356.6144 - val_loss: 21724.8359
Epoch 199/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 350.2356 - val_loss: 21725.9434
Epoch 200/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 339.1218 - val_loss: 21686.5859
Epoch 201/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 336.9100 - val_loss: 21694.6973
Epoch 202/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 358.4460 - val_loss: 21629.7891
Epoch 203/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 358.4482 - val_loss: 21

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1314.6504 - val_loss: 24030.9023
Epoch 258/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1521.1797 - val_loss: 24139.4473
Epoch 259/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1909.2141 - val_loss: 24299.1836
Epoch 260/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2358.4946 - val_loss: 24453.2871
Epoch 261/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2576.2073 - val_loss: 25540.9863
Epoch 262/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2407.5732 - val_loss: 28875.4375
Epoch 263/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2609.3474 - val_loss: 31549.2539
Epoch 264/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3268.7295 - val_loss: 30568.9062
Epoch 265/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3422.5308 - val_loss: 28119.5000
Epoch 266/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2893.4548 - val_loss: 27561.9102
Epoch 267/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2613.0427 - v

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 286.8333 - val_loss: 25439.3594
Epoch 322/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 281.8734 - val_loss: 25657.6602
Epoch 323/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 284.9926 - val_loss: 25781.9043
Epoch 324/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 281.7469 - val_loss: 25948.5586
Epoch 325/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 301.2281 - val_loss: 25866.4336
Epoch 326/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 295.5399 - val_loss: 26074.5586
Epoch 327/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 297.5832 - val_loss: 25979.5215
Epoch 328/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 301.3081 - val_loss: 25990.2930
Epoch 329/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 299.8282 - val_loss: 25772.2480
Epoch 330/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 294.2406 - val_loss: 25526.2812
Epoch 331/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 296.8863 - val_loss: 25

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4281.3047 - val_loss: 52340.3711
Epoch 386/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4047.3372 - val_loss: 51555.6523
Epoch 387/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3829.8801 - val_loss: 50393.7617
Epoch 388/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3781.7046 - val_loss: 48250.8867
Epoch 389/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3810.5266 - val_loss: 45042.3789
Epoch 390/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3828.7375 - val_loss: 40834.0391
Epoch 391/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3864.2668 - val_loss: 35557.7578
Epoch 392/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3895.1379 - val_loss: 29519.0273
Epoch 393/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4032.4243 - val_loss: 24706.0215
Epoch 394/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4300.0249 - val_loss: 21902.7285
Epoch 395/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4808.9102 - v

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 132.6245 - val_loss: 23711.5059
Epoch 450/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 130.2601 - val_loss: 23761.9336
Epoch 451/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 129.5351 - val_loss: 23784.7031
Epoch 452/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 127.7132 - val_loss: 23824.2148
Epoch 453/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 124.8723 - val_loss: 23843.6699
Epoch 454/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 120.2743 - val_loss: 23873.3789
Epoch 455/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 119.6606 - val_loss: 23951.8887
Epoch 456/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 116.0466 - val_loss: 23957.1777
Epoch 457/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 114.6622 - val_loss: 24023.2109
Epoch 458/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 112.6617 - val_loss: 24041.2344
Epoch 459/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 109.7356 - val_loss: 24

In [22]:
if predictions.ndim != 1:
    predictions = predictions.flatten()

# Check if 'y_test' is not 1-dimensional and reshape if necessary
if y_test.ndim != 1:
    y_test = y_test.flatten()
    
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
print(results)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(mae, 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

     Prediction      Actual
0    198.660278  119.793729
1    432.001312  359.563884
2    209.419785  168.934525
3    317.022522  355.505368
4    596.151550  433.151080
..          ...         ...
112  738.340027  487.957347
113  252.905655  216.123044
114  441.012909  425.953734
115  338.890381  436.229287
116  438.448395  388.424770

[117 rows x 2 columns]
Mean Absolute Error (MAE): 81.83923697943864
Mean Squared Error (MSE): 12975.789068175483
Root Mean Squared Error (RMSE): 113.91132107115378
R-squared (R²): 0.7702935388699319
Mean Absolute Error: 81.84 degrees.
Accuracy: 71.78 %.


C:\Users\EJ-PC\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [23]:
# build model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)  # No activation for regression
])

# compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# train model
history = model.fit(X_train, y_train, epochs=1000, batch_size=64, validation_split=0.15)

# evaluate the model
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

#predict
predictions = model.predict(X_test)

Epoch 1/1000


C:\Users\EJ-PC\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 221886.3281 - val_loss: 224662.7500
Epoch 2/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 211575.7500 - val_loss: 194931.6719
Epoch 3/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 172716.6094 - val_loss: 130136.8516
Epoch 4/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 104139.0000 - val_loss: 87582.7422
Epoch 5/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 79187.3047 - val_loss: 63571.6445
Epoch 6/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 67138.4375 - val_loss: 56869.3164
Epoch 7/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 61122.4922 - val_loss: 53350.0781
Epoch 8/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 57141.2539 - val_loss: 49582.8398
Epoch 9/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 53013.4180 - val_loss: 46445.7461
Epoch 10/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 49964.1523 - val_loss: 43786.3438
Epoch 11/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4730

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 10168.4912 - val_loss: 21083.7188
Epoch 65/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9940.1709 - val_loss: 21081.9570
Epoch 66/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9732.5127 - val_loss: 21114.9980
Epoch 67/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9514.2383 - val_loss: 21160.8145
Epoch 68/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9313.4248 - val_loss: 21172.4785
Epoch 69/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9115.0586 - val_loss: 21195.6543
Epoch 70/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8921.2529 - val_loss: 21169.1016
Epoch 71/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8723.3613 - val_loss: 21263.4121
Epoch 72/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8561.7744 - val_loss: 21275.4199
Epoch 73/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8363.5430 - val_loss: 21216.5703
Epoch 74/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8161.0366 - 

Epoch 128/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8746.3584 - val_loss: 26127.8770
Epoch 129/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6244.7725 - val_loss: 23252.5098
Epoch 130/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4797.2842 - val_loss: 22923.8340
Epoch 131/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4238.9463 - val_loss: 22694.2246
Epoch 132/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3846.5513 - val_loss: 22229.2832
Epoch 133/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3355.2891 - val_loss: 22081.5254
Epoch 134/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2811.0535 - val_loss: 22497.2363
Epoch 135/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2460.3691 - val_loss: 23137.0488
Epoch 136/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2315.5364 - val_loss: 23760.7129
Epoch 137/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2283.8115 - val_loss: 24294.8809
Epoch 138/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/s

...

Epoch 702/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 828.8811 - val_loss: 26075.8105
Epoch 703/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 608.9056 - val_loss: 23905.2832
Epoch 704/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 362.1347 - val_loss: 22109.3203
Epoch 705/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 322.5182 - val_loss: 21305.2539
Epoch 706/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 539.8331 - val_loss: 21348.7324
Epoch 707/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 842.4097 - val_loss: 21689.6230
Epoch 708/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 935.6413 - val_loss: 22147.2500
Epoch 709/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 765.5201 - val_loss: 22898.0801
Epoch 710/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 522.5511 - val_loss: 24168.1582
Epoch 711/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 407.0462 - val_loss: 25844.3828
Epoch 712/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss

Epoch 766/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 755.8501 - val_loss: 22362.8203
Epoch 767/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 498.2337 - val_loss: 23965.6309
Epoch 768/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 355.5305 - val_loss: 26197.0879
Epoch 769/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 464.3638 - val_loss: 27694.5684
Epoch 770/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 717.5211 - val_loss: 27213.5527
Epoch 771/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 761.1698 - val_loss: 24968.8828
Epoch 772/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 535.5389 - val_loss: 22816.8340
Epoch 773/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 356.5227 - val_loss: 21581.8633
Epoch 774/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 438.9163 - val_loss: 21441.4414
Epoch 775/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 705.0439 - val_loss: 21745.7656
Epoch 776/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 451.4568 - val_loss: 23084.8613
Epoch 831/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 301.5034 - val_loss: 24037.2031
Epoch 832/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 322.6231 - val_loss: 24768.8223
Epoch 833/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 379.4960 - val_loss: 25055.0723
Epoch 834/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 380.5729 - val_loss: 25129.2324
Epoch 835/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 333.8006 - val_loss: 24950.7598
Epoch 836/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 326.4989 - val_loss: 24175.3066
Epoch 837/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 368.0388 - val_loss: 23214.1660
Epoch 838/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 383.7892 - val_loss: 22541.4746
Epoch 839/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 332.1232 - val_loss: 22519.6152
Epoch 840/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 297.8092 - va

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 409.0748 - val_loss: 22590.7207
Epoch 895/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 287.0981 - val_loss: 21729.1934
Epoch 896/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 388.5179 - val_loss: 21587.6406
Epoch 897/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 592.9871 - val_loss: 21662.2031
Epoch 898/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 631.7936 - val_loss: 22118.1660
Epoch 899/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 452.7646 - val_loss: 23494.0996
Epoch 900/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 295.3580 - val_loss: 25516.5879
Epoch 901/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 362.4132 - val_loss: 26989.4590
Epoch 902/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 569.5128 - val_loss: 26864.6172
Epoch 903/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 661.5760 - val_loss: 24977.5215
Epoch 904/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 481.6647 - va

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 496.9476 - val_loss: 23239.3828
Epoch 959/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 263.2498 - val_loss: 25111.9980
Epoch 960/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 283.0008 - val_loss: 26652.1465
Epoch 961/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 483.7010 - val_loss: 26561.2500
Epoch 962/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 556.7469 - val_loss: 24732.9668
Epoch 963/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 398.7435 - val_loss: 22599.7402
Epoch 964/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 258.6231 - val_loss: 21531.2871
Epoch 965/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 336.3380 - val_loss: 21530.2598
Epoch 966/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 556.9209 - val_loss: 21889.4316
Epoch 967/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 633.8624 - val_loss: 22328.5527
Epoch 968/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 472.5153 - va

In [24]:
if predictions.ndim != 1:
    predictions = predictions.flatten()

# Check if 'y_test' is not 1-dimensional and reshape if necessary
if y_test.ndim != 1:
    y_test = y_test.flatten()
    
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
print(results)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(mae, 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

     Prediction      Actual
0    197.770737  119.793729
1    255.742294  359.563884
2    322.107605  168.934525
3    341.519318  355.505368
4    450.812744  433.151080
..          ...         ...
112  663.453003  487.957347
113  223.211655  216.123044
114  356.171051  425.953734
115  354.241089  436.229287
116  459.355530  388.424770

[117 rows x 2 columns]
Mean Absolute Error (MAE): 83.20141333297829
Mean Squared Error (MSE): 12651.030685271136
Root Mean Squared Error (RMSE): 112.4768006536065
R-squared (R²): 0.7760426380936735
Mean Absolute Error: 83.2 degrees.
Accuracy: 71.31 %.


C:\Users\EJ-PC\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [25]:
# build model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)  # No activation for regression
])

# compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# train model
history = model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_split=0.15)

# evaluate the model
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

#predict
predictions = model.predict(X_test)

Epoch 1/1000


C:\Users\EJ-PC\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 218643.0156 - val_loss: 211470.9219
Epoch 2/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 169729.8750 - val_loss: 98304.4062
Epoch 3/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 83303.6250 - val_loss: 62816.4805
Epoch 4/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 63966.2305 - val_loss: 54917.6484
Epoch 5/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 56916.3359 - val_loss: 49396.5391
Epoch 6/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 51051.6875 - val_loss: 44434.1367
Epoch 7/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 46362.7617 - val_loss: 41100.1719
Epoch 8/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 43043.1016 - val_loss: 39182.1211
Epoch 9/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 40817.2812 - val_loss: 37366.3242
Epoch 10/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 38953.9180 - val_loss: 36325.9258
Epoch 11/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 37581.16

Epoch 65/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4826.4863 - val_loss: 20332.5371
Epoch 66/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4626.4111 - val_loss: 20451.7402
Epoch 67/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4524.0508 - val_loss: 20414.2012
Epoch 68/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4339.3887 - val_loss: 20510.2500
Epoch 69/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4239.4155 - val_loss: 20506.4121
Epoch 70/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4092.3499 - val_loss: 20552.0117
Epoch 71/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4006.8489 - val_loss: 20546.1680
Epoch 72/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3867.3071 - val_loss: 20492.5684
Epoch 73/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3776.3171 - val_loss: 20427.8945
Epoch 74/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3638.2815 - val_loss: 20399.9492
Epoch 75/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss:

Epoch 129/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1945.7274 - val_loss: 20592.6973
Epoch 130/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1714.4366 - val_loss: 20194.0098
Epoch 131/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1469.8251 - val_loss: 19978.3496
Epoch 132/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1310.8063 - val_loss: 19871.2695
Epoch 133/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1193.0408 - val_loss: 19809.0410
Epoch 134/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1125.6515 - val_loss: 19680.0742
Epoch 135/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1066.0806 - val_loss: 19635.9102
Epoch 136/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1036.5264 - val_loss: 19602.4180
Epoch 137/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 973.7542 - val_loss: 19459.6309
Epoch 138/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 941.4835 - val_loss: 19433.9629
Epoch 139/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/ste

...

Epoch 703/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1287.3740 - val_loss: 20280.0586
Epoch 704/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1286.4873 - val_loss: 20594.0098
Epoch 705/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1101.1709 - val_loss: 20779.1660
Epoch 706/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 927.8630 - val_loss: 21506.5820
Epoch 707/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 857.0499 - val_loss: 24186.3398
Epoch 708/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1000.0681 - val_loss: 27562.8652
Epoch 709/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1182.8690 - val_loss: 28270.7773
Epoch 710/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1139.6514 - val_loss: 25738.3945
Epoch 711/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 846.3639 - val_loss: 22227.1641
Epoch 712/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 735.1038 - val_loss: 19744.1895
Epoch 713/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 

Epoch 767/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 860.9961 - val_loss: 19281.4141
Epoch 768/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 889.8788 - val_loss: 19587.8496
Epoch 769/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 895.4093 - val_loss: 20059.5430
Epoch 770/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 806.9501 - val_loss: 20013.2598
Epoch 771/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 739.2499 - val_loss: 21698.9023
Epoch 772/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 678.7329 - val_loss: 24129.0820
Epoch 773/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 816.0186 - val_loss: 26835.9844
Epoch 774/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 976.2503 - val_loss: 27468.3965
Epoch 775/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 982.4072 - val_loss: 25410.6797
Epoch 776/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 847.8504 - val_loss: 22900.1309
Epoch 777/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 793.7347 - val_loss: 19494.6680
Epoch 832/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 728.4034 - val_loss: 20574.8340
Epoch 833/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 612.3663 - val_loss: 23132.9473
Epoch 834/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 675.5922 - val_loss: 25997.3848
Epoch 835/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 842.2169 - val_loss: 26783.7246
Epoch 836/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 867.2739 - val_loss: 24369.8535
Epoch 837/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 762.3719 - val_loss: 21286.1055
Epoch 838/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 645.6803 - val_loss: 18820.4512
Epoch 839/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 596.9286 - val_loss: 18257.1953
Epoch 840/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 633.7173 - val_loss: 18508.5273
Epoch 841/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 749.2092 - va

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 723.8328 - val_loss: 21902.2910
Epoch 896/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 656.9393 - val_loss: 19389.6562
Epoch 897/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 590.1293 - val_loss: 17858.6641
Epoch 898/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 582.1069 - val_loss: 17808.0469
Epoch 899/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 617.6320 - val_loss: 18451.4102
Epoch 900/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 705.2404 - val_loss: 18590.6602
Epoch 901/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 687.5643 - val_loss: 19352.2656
Epoch 902/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 674.8501 - val_loss: 20788.8242
Epoch 903/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 576.3741 - val_loss: 23198.8125
Epoch 904/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 605.6353 - val_loss: 25121.3965
Epoch 905/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 709.7123 - va

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 612.6536 - val_loss: 19843.2500
Epoch 960/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 550.9038 - val_loss: 22097.4062
Epoch 961/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 580.6459 - val_loss: 23953.3281
Epoch 962/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 647.3965 - val_loss: 24180.2949
Epoch 963/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 659.8132 - val_loss: 22463.3203
Epoch 964/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 610.6002 - val_loss: 20419.3945
Epoch 965/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 550.4576 - val_loss: 18856.8438
Epoch 966/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 554.7736 - val_loss: 18217.3984
Epoch 967/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 555.3171 - val_loss: 17973.1660
Epoch 968/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 548.7637 - val_loss: 18104.3242
Epoch 969/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 590.2308 - va

In [26]:
if predictions.ndim != 1:
    predictions = predictions.flatten()

# Check if 'y_test' is not 1-dimensional and reshape if necessary
if y_test.ndim != 1:
    y_test = y_test.flatten()
    
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
print(results)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(mae, 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

     Prediction      Actual
0    157.552322  119.793729
1    285.615112  359.563884
2    370.533386  168.934525
3    382.574402  355.505368
4    521.935669  433.151080
..          ...         ...
112  694.018677  487.957347
113  384.436890  216.123044
114  423.625610  425.953734
115  357.813293  436.229287
116  388.941833  388.424770

[117 rows x 2 columns]
Mean Absolute Error (MAE): 83.62170541486563
Mean Squared Error (MSE): 13321.59982526519
Root Mean Squared Error (RMSE): 115.41923507485738
R-squared (R²): 0.7641717558466082
Mean Absolute Error: 83.62 degrees.
Accuracy: 72.15 %.


C:\Users\EJ-PC\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [27]:
# Assuming 'model' is your Keras model variable
#model.save('80model.h5')  # Save the model as an HDF5 file


In [28]:
# build model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)  # No activation for regression
])

# compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# train model
history = model.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.15)

# evaluate the model
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

#predict
predictions = model.predict(X_test)

Epoch 1/500


C:\Users\EJ-PC\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 221648.3438 - val_loss: 221268.5469
Epoch 2/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 204220.1406 - val_loss: 159028.7656
Epoch 3/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 129148.6797 - val_loss: 95647.7891
Epoch 4/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 84897.0938 - val_loss: 61431.1250
Epoch 5/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 66114.0469 - val_loss: 53658.6914
Epoch 6/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 59817.6641 - val_loss: 49590.3477
Epoch 7/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 53618.2383 - val_loss: 44575.3164
Epoch 8/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 49587.5039 - val_loss: 40890.7500
Epoch 9/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 45977.6289 - val_loss: 38621.8555
Epoch 10/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 43349.4414 - val_loss: 36614.7500
Epoch 11/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 41156.2109 - va

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4228.5596 - val_loss: 23452.3613
Epoch 66/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3705.5212 - val_loss: 23583.5703
Epoch 67/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3403.7852 - val_loss: 24098.2734
Epoch 68/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3236.2139 - val_loss: 24568.7969
Epoch 69/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3131.5212 - val_loss: 24950.7402
Epoch 70/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3029.9136 - val_loss: 25440.7344
Epoch 71/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2976.1753 - val_loss: 25818.8125
Epoch 72/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2924.6208 - val_loss: 26235.5469
Epoch 73/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2876.0579 - val_loss: 26470.2090
Epoch 74/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2820.7036 - val_loss: 26715.1934
Epoch 75/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2757.8582 - val_loss: 2

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1214.1991 - val_loss: 25803.2676
Epoch 130/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1126.7562 - val_loss: 24849.4219
Epoch 131/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1010.5233 - val_loss: 23706.6641
Epoch 132/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 911.1614 - val_loss: 22713.2363
Epoch 133/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 865.2270 - val_loss: 21944.2031
Epoch 134/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 924.3613 - val_loss: 21591.3457
Epoch 135/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1091.2227 - val_loss: 21625.9746
Epoch 136/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1358.1620 - val_loss: 22042.9121
Epoch 137/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1651.7781 - val_loss: 22475.5469
Epoch 138/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1930.8296 - val_loss: 22806.0020
Epoch 139/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2138.6570 - val_

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1178.1292 - val_loss: 21126.9902
Epoch 194/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1584.3682 - val_loss: 21783.7422
Epoch 195/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1954.2832 - val_loss: 21911.5000
Epoch 196/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1945.9840 - val_loss: 21677.8770
Epoch 197/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1657.6862 - val_loss: 21969.6406
Epoch 198/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1311.6068 - val_loss: 23437.2363
Epoch 199/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1149.0398 - val_loss: 25949.5684
Epoch 200/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1266.3092 - val_loss: 27899.3281
Epoch 201/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1576.6155 - val_loss: 28152.9121
Epoch 202/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1800.5720 - val_loss: 26455.1699
Epoch 203/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1689.1852 - v

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1397.0817 - val_loss: 21265.7676
Epoch 258/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1158.7064 - val_loss: 22080.0859
Epoch 259/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 935.8835 - val_loss: 23834.6816
Epoch 260/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 902.7759 - val_loss: 26012.0371
Epoch 261/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1132.6261 - val_loss: 26855.8301
Epoch 262/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1381.0775 - val_loss: 25535.9863
Epoch 263/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1356.1252 - val_loss: 23262.5312
Epoch 264/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1058.6829 - val_loss: 21217.9434
Epoch 265/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 856.0430 - val_loss: 20616.6719
Epoch 266/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 976.2448 - val_loss: 20925.4512
Epoch 267/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1285.8848 - val_l

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 832.8492 - val_loss: 20399.2637
Epoch 322/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 893.7476 - val_loss: 20891.7656
Epoch 323/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1230.6991 - val_loss: 21180.1660
Epoch 324/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1387.4896 - val_loss: 21006.2402
Epoch 325/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1216.4983 - val_loss: 21415.9590
Epoch 326/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 953.2448 - val_loss: 22954.9844
Epoch 327/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 838.2853 - val_loss: 25161.0879
Epoch 328/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 961.4927 - val_loss: 26440.9062
Epoch 329/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1237.1425 - val_loss: 25407.8359
Epoch 330/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1275.9866 - val_loss: 22793.4219
Epoch 331/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 987.3199 - val_los

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 914.1247 - val_loss: 20947.4141
Epoch 386/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1158.7091 - val_loss: 20995.2598
Epoch 387/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1138.8864 - val_loss: 21105.3301
Epoch 388/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 952.4123 - val_loss: 22340.1816
Epoch 389/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 819.7539 - val_loss: 24475.3965
Epoch 390/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 854.2089 - val_loss: 25802.3652
Epoch 391/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1057.1879 - val_loss: 25071.1230
Epoch 392/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1173.9570 - val_loss: 22547.4863
Epoch 393/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 943.8912 - val_loss: 20612.3457
Epoch 394/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 726.9172 - val_loss: 20199.6035
Epoch 395/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 860.3698 - val_loss

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 767.7987 - val_loss: 25115.2344
Epoch 450/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 898.3086 - val_loss: 24913.7910
Epoch 451/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1025.5361 - val_loss: 22702.6309
Epoch 452/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 835.3221 - val_loss: 20515.9121
Epoch 453/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 624.9111 - val_loss: 19969.4902
Epoch 454/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 737.8361 - val_loss: 20423.9121
Epoch 455/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 938.9241 - val_loss: 20629.4082
Epoch 456/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 916.0129 - val_loss: 20881.1973
Epoch 457/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 770.5752 - val_loss: 22146.4121
Epoch 458/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 689.5659 - val_loss: 24110.4473
Epoch 459/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 751.9447 - val_loss: 2

In [29]:
if predictions.ndim != 1:
    predictions = predictions.flatten()

# Check if 'y_test' is not 1-dimensional and reshape if necessary
if y_test.ndim != 1:
    y_test = y_test.flatten()
    
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
print(results)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(mae, 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

     Prediction      Actual
0    151.967834  119.793729
1    332.645050  359.563884
2    148.031342  168.934525
3    367.770416  355.505368
4    474.415375  433.151080
..          ...         ...
112  666.455811  487.957347
113  188.250671  216.123044
114  378.390350  425.953734
115  256.703033  436.229287
116  414.770599  388.424770

[117 rows x 2 columns]
Mean Absolute Error (MAE): 77.40660657222531
Mean Squared Error (MSE): 11279.235381275597
Root Mean Squared Error (RMSE): 106.2037446669165
R-squared (R²): 0.8003271145921782
Mean Absolute Error: 77.41 degrees.
Accuracy: 72.56 %.


C:\Users\EJ-PC\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [30]:
# build model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)  # No activation for regression
])

# compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# train model
history = model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_split=0.15)

# evaluate the model
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

#predict
predictions = model.predict(X_test)

Epoch 1/1000


C:\Users\EJ-PC\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 215216.8594 - val_loss: 157648.9219
Epoch 2/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 107562.7188 - val_loss: 64647.4883
Epoch 3/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 63478.8633 - val_loss: 53453.9688
Epoch 4/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 53756.5430 - val_loss: 46009.4609
Epoch 5/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 46925.4219 - val_loss: 42935.8203
Epoch 6/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 43198.8477 - val_loss: 39619.0391
Epoch 7/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 39966.6406 - val_loss: 37728.2695
Epoch 8/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 37653.3359 - val_loss: 36440.4297
Epoch 9/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 35507.6562 - val_loss: 34643.7188
Epoch 10/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 33116.7734 - val_loss: 32543.5820
Epoch 11/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 30515.49

Epoch 65/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2419.3208 - val_loss: 23727.2285
Epoch 66/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2288.8240 - val_loss: 23811.4648
Epoch 67/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3313.9358 - val_loss: 24895.4648
Epoch 68/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3594.7878 - val_loss: 24867.5859
Epoch 69/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2791.9260 - val_loss: 27028.4004
Epoch 70/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3790.2651 - val_loss: 31617.9785
Epoch 71/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7945.6045 - val_loss: 39148.7695
Epoch 72/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 15621.9307 - val_loss: 26454.7930
Epoch 73/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 17514.9375 - val_loss: 33133.4688
Epoch 74/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 11979.4180 - val_loss: 20451.6348
Epoch 75/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - lo

Epoch 129/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7366.0317 - val_loss: 22441.5391
Epoch 130/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4032.4404 - val_loss: 23058.2871
Epoch 131/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3030.9905 - val_loss: 24950.8301
Epoch 132/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3004.4937 - val_loss: 24788.0352
Epoch 133/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2907.0476 - val_loss: 24051.6133
Epoch 134/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2366.2107 - val_loss: 23654.8672
Epoch 135/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2060.4700 - val_loss: 24053.1074
Epoch 136/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2019.6061 - val_loss: 24441.2559
Epoch 137/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2161.9116 - val_loss: 24811.7129
Epoch 138/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2396.4644 - val_loss: 24995.2148
Epoch 139/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/s

...

Epoch 700/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 673.8733 - val_loss: 14144.5703
Epoch 701/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 779.2873 - val_loss: 15055.6592
Epoch 702/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 983.7229 - val_loss: 14845.3252
Epoch 703/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 928.6053 - val_loss: 14730.7451
Epoch 704/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 793.8951 - val_loss: 15640.6250
Epoch 705/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 672.6521 - val_loss: 17688.7598
Epoch 706/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 999.4632 - val_loss: 16678.5508
Epoch 707/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 986.7252 - val_loss: 14759.0703
Epoch 708/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 726.9071 - val_loss: 14621.2129
Epoch 709/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 731.9640 - val_loss: 15469.7344
Epoch 710/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss

Epoch 764/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1014.8818 - val_loss: 15780.9326
Epoch 765/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 700.4417 - val_loss: 14661.1719
Epoch 766/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 514.5552 - val_loss: 15437.4473
Epoch 767/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 735.1996 - val_loss: 16541.3594
Epoch 768/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 906.5671 - val_loss: 15694.2969
Epoch 769/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 837.2402 - val_loss: 15422.4092
Epoch 770/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 583.4157 - val_loss: 16625.3008
Epoch 771/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 741.8601 - val_loss: 17961.4316
Epoch 772/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 999.0215 - val_loss: 16455.0762
Epoch 773/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 760.9323 - val_loss: 14861.1465
Epoch 774/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - los

Epoch 828/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 997.7646 - val_loss: 15714.9688
Epoch 829/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 725.4853 - val_loss: 14677.2002
Epoch 830/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 561.5647 - val_loss: 15881.0996
Epoch 831/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 804.0740 - val_loss: 16314.9297
Epoch 832/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 945.7612 - val_loss: 15485.7217
Epoch 833/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 707.9379 - val_loss: 16087.6074
Epoch 834/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 545.4649 - val_loss: 18226.2246
Epoch 835/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 808.3314 - val_loss: 17625.9102
Epoch 836/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 889.0176 - val_loss: 15465.4707
Epoch 837/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 575.8766 - val_loss: 14650.0986
Epoch 838/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 483.7711 - val_loss: 16096.8301
Epoch 893/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 738.8742 - val_loss: 15943.8379
Epoch 894/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 839.8939 - val_loss: 15036.8486
Epoch 895/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 701.9506 - val_loss: 16294.4971
Epoch 896/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 541.5653 - val_loss: 18463.8965
Epoch 897/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 855.7198 - val_loss: 18247.3477
Epoch 898/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 965.6302 - val_loss: 15918.9609
Epoch 899/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 611.2250 - val_loss: 15565.1074
Epoch 900/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 567.4448 - val_loss: 16683.5977
Epoch 901/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 813.8290 - val_loss: 16209.7314
Epoch 902/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 783.7297 - va

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 708.7075 - val_loss: 15663.7754
Epoch 957/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 540.0045 - val_loss: 17412.4062
Epoch 958/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 468.6939 - val_loss: 19346.6289
Epoch 959/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 676.1269 - val_loss: 17443.3555
Epoch 960/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 598.2889 - val_loss: 15489.1143
Epoch 961/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 427.5750 - val_loss: 15440.9346
Epoch 962/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 506.8971 - val_loss: 15950.1465
Epoch 963/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 626.0367 - val_loss: 15929.4473
Epoch 964/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 595.2426 - val_loss: 16222.7480
Epoch 965/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 415.2904 - val_loss: 17334.3711
Epoch 966/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 555.7728 - va

In [31]:
if predictions.ndim != 1:
    predictions = predictions.flatten()

# Check if 'y_test' is not 1-dimensional and reshape if necessary
if y_test.ndim != 1:
    y_test = y_test.flatten()
    
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
print(results)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(mae, 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

     Prediction      Actual
0    201.498276  119.793729
1    313.315247  359.563884
2    262.460205  168.934525
3    279.305237  355.505368
4    515.239563  433.151080
..          ...         ...
112  586.556763  487.957347
113  252.801804  216.123044
114  414.705688  425.953734
115  276.510803  436.229287
116  333.316772  388.424770

[117 rows x 2 columns]
Mean Absolute Error (MAE): 82.44008678256776
Mean Squared Error (MSE): 12963.215670783704
Root Mean Squared Error (RMSE): 113.85611828436672
R-squared (R²): 0.770516121913173
Mean Absolute Error: 82.44 degrees.
Accuracy: 73.12 %.


C:\Users\EJ-PC\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [32]:
# build model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)  # No activation for regression
])

# compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# train model
history = model.fit(X_train, y_train, epochs=1000, batch_size=16, validation_split=0.15)

# evaluate the model
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

#predict
predictions = model.predict(X_test)

Epoch 1/1000


C:\Users\EJ-PC\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 207268.9531 - val_loss: 145768.5938
Epoch 2/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 96794.5469 - val_loss: 62556.6445
Epoch 3/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 60800.3945 - val_loss: 54170.5234
Epoch 4/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 52997.9258 - val_loss: 46990.5508
Epoch 5/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 47169.2227 - val_loss: 43038.0234
Epoch 6/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 43555.9258 - val_loss: 40772.0273
Epoch 7/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 41255.7227 - val_loss: 39092.4492
Epoch 8/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 39451.4336 - val_loss: 37937.7305
Epoch 9/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 38098.0000 - val_loss: 37027.8047
Epoch 10/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 36815.2734 - val_loss: 36749.1680
Epoch 11/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 35865.449

Epoch 65/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4129.7930 - val_loss: 26908.4648
Epoch 66/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3972.9285 - val_loss: 26827.3848
Epoch 67/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3808.1436 - val_loss: 26831.7930
Epoch 68/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3679.5850 - val_loss: 26676.2773
Epoch 69/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3545.8440 - val_loss: 26886.0195
Epoch 70/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3437.3813 - val_loss: 26724.7930
Epoch 71/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3272.8938 - val_loss: 27065.8398
Epoch 72/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3218.2815 - val_loss: 26970.8691
Epoch 73/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3101.9167 - val_loss: 27028.4844
Epoch 74/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3026.8826 - val_loss: 26897.2773
Epoch 75/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss:

Epoch 129/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1354.5516 - val_loss: 30587.8848
Epoch 130/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1404.9120 - val_loss: 29493.5840
Epoch 131/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1328.1594 - val_loss: 30023.7031
Epoch 132/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1378.9211 - val_loss: 29377.5410
Epoch 133/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1342.8618 - val_loss: 29605.9746
Epoch 134/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1381.1713 - val_loss: 28552.9883
Epoch 135/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1374.5747 - val_loss: 29165.0098
Epoch 136/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1479.8829 - val_loss: 28018.6934
Epoch 137/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1463.7408 - val_loss: 28144.1973
Epoch 138/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1567.8280 - val_loss: 26645.1992
Epoch 139/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/s

...

56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 494.5824 - val_loss: 18671.3164
Epoch 705/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 479.3332 - val_loss: 17869.8047
Epoch 706/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 642.3851 - val_loss: 17638.7949
Epoch 707/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 661.7037 - val_loss: 19196.0508
Epoch 708/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 424.4474 - val_loss: 21356.0117
Epoch 709/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 451.6859 - val_loss: 22489.7773
Epoch 710/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 645.6278 - val_loss: 22379.0547
Epoch 711/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 660.2338 - val_loss: 20179.1602
Epoch 712/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 411.6910 - val_loss: 18989.9473
Epoch 713/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 527.9485 - val_loss: 18134.3770
Epoch 714/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 568.3549 - va

56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 429.5577 - val_loss: 17607.9492
Epoch 769/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 501.9167 - val_loss: 18271.9824
Epoch 770/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 511.7434 - val_loss: 19375.9414
Epoch 771/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 386.9955 - val_loss: 21164.6191
Epoch 772/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 435.0676 - val_loss: 22521.8008
Epoch 773/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 595.7589 - val_loss: 22011.0000
Epoch 774/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 743.7628 - val_loss: 19967.3105
Epoch 775/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 539.5278 - val_loss: 18687.1562
Epoch 776/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 459.6179 - val_loss: 17761.5742
Epoch 777/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 534.6633 - val_loss: 17831.6816
Epoch 778/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 612.8770 - va

56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 609.2124 - val_loss: 22733.1582
Epoch 833/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 719.9062 - val_loss: 20251.4102
Epoch 834/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 497.5418 - val_loss: 18361.4785
Epoch 835/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 417.8626 - val_loss: 17177.7129
Epoch 836/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 574.6846 - val_loss: 17282.3789
Epoch 837/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 539.7283 - val_loss: 19567.9863
Epoch 838/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 432.7332 - val_loss: 22310.6504
Epoch 839/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 541.9833 - val_loss: 23364.3887
Epoch 840/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 746.2852 - val_loss: 20412.7188
Epoch 841/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 530.3526 - val_loss: 18662.8008
Epoch 842/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 435.0827 - va

56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 393.1416 - val_loss: 19271.0547
Epoch 897/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 411.1575 - val_loss: 20395.1504
Epoch 898/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 543.5535 - val_loss: 19459.3008
Epoch 899/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 425.6954 - val_loss: 18518.5645
Epoch 900/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 309.5996 - val_loss: 17886.6309
Epoch 901/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 387.0747 - val_loss: 17759.2090
Epoch 902/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 503.8047 - val_loss: 17685.9551
Epoch 903/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 445.7131 - val_loss: 18529.9336
Epoch 904/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 395.1464 - val_loss: 20707.7812
Epoch 905/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 518.2640 - val_loss: 20280.3457
Epoch 906/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 617.7475 - va

56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 317.7461 - val_loss: 21285.8008
Epoch 961/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 422.4541 - val_loss: 22166.6426
Epoch 962/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 626.6623 - val_loss: 20211.9512
Epoch 963/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 486.1118 - val_loss: 18820.9297
Epoch 964/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 421.8900 - val_loss: 17398.5117
Epoch 965/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 534.7665 - val_loss: 17049.3125
Epoch 966/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 458.9476 - val_loss: 18462.3086
Epoch 967/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 302.5146 - val_loss: 21224.1914
Epoch 968/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 430.9518 - val_loss: 20661.0059
Epoch 969/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 544.0989 - val_loss: 18799.8535
Epoch 970/1000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 388.5090 - va

In [33]:
if predictions.ndim != 1:
    predictions = predictions.flatten()

# Check if 'y_test' is not 1-dimensional and reshape if necessary
if y_test.ndim != 1:
    y_test = y_test.flatten()
    
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
print(results)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(mae, 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

     Prediction      Actual
0    165.528702  119.793729
1    296.722687  359.563884
2    202.274277  168.934525
3    347.328094  355.505368
4    454.991425  433.151080
..          ...         ...
112  677.056763  487.957347
113  259.608826  216.123044
114  449.531036  425.953734
115  410.426025  436.229287
116  402.885010  388.424770

[117 rows x 2 columns]
Mean Absolute Error (MAE): 82.24837233014995
Mean Squared Error (MSE): 13369.036884290743
Root Mean Squared Error (RMSE): 115.62455139065726
R-squared (R²): 0.7633319919680551
Mean Absolute Error: 82.25 degrees.
Accuracy: 73.45 %.


C:\Users\EJ-PC\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [34]:
# build model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='swish', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='swish'),
    tf.keras.layers.Dense(128, activation='swish'),
    tf.keras.layers.Dense(1)  # No activation for regression
])

# compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# train model
history = model.fit(X_train, y_train, epochs=500, batch_size=32, validation_split=0.15)

# evaluate the model
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

#predict
predictions = model.predict(X_test)

Epoch 1/500


C:\Users\EJ-PC\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 220224.4688 - val_loss: 225169.0469
Epoch 2/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 192719.7031 - val_loss: 143492.0938
Epoch 3/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 111545.8047 - val_loss: 78416.4609
Epoch 4/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 78348.8594 - val_loss: 65632.5156
Epoch 5/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 68595.9688 - val_loss: 58585.6055
Epoch 6/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 61032.0234 - val_loss: 52620.4062
Epoch 7/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 54870.0312 - val_loss: 47169.3008
Epoch 8/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 49363.0000 - val_loss: 43455.8281
Epoch 9/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 45043.8047 - val_loss: 40254.4102
Epoch 10/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 41379.6406 - val_loss: 37638.2734
Epoch 11/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 38347.5195 - val

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5243.1050 - val_loss: 20240.9961
Epoch 66/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5097.8784 - val_loss: 20258.5117
Epoch 67/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4948.1631 - val_loss: 20227.2910
Epoch 68/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4812.2549 - val_loss: 20219.2070
Epoch 69/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4661.7529 - val_loss: 20191.2461
Epoch 70/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4530.9097 - val_loss: 20186.1035
Epoch 71/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4387.5195 - val_loss: 20134.6953
Epoch 72/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4254.7319 - val_loss: 20087.3945
Epoch 73/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4115.9624 - val_loss: 20105.1152
Epoch 74/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4000.1057 - val_loss: 20022.9336
Epoch 75/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3877.4331 - val_loss: 2

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 883.1175 - val_loss: 18760.9375
Epoch 130/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 812.9521 - val_loss: 19666.3340
Epoch 131/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 833.8854 - val_loss: 18989.7949
Epoch 132/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 796.9720 - val_loss: 19493.8770
Epoch 133/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 774.0477 - val_loss: 19369.0312
Epoch 134/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 849.6698 - val_loss: 19362.7598
Epoch 135/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 818.2944 - val_loss: 20247.6719
Epoch 136/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 895.6627 - val_loss: 19166.5117
Epoch 137/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 881.7780 - val_loss: 19965.7754
Epoch 138/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 842.4223 - val_loss: 20513.3086
Epoch 139/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 985.9431 - val_loss: 19

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 688.7982 - val_loss: 20862.4844
Epoch 194/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 649.4072 - val_loss: 20938.4668
Epoch 195/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 625.2485 - val_loss: 21052.2402
Epoch 196/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 616.8481 - val_loss: 21145.0879
Epoch 197/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 610.1962 - val_loss: 21274.2188
Epoch 198/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 613.7703 - val_loss: 21397.2148
Epoch 199/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 615.4427 - val_loss: 21570.4023
Epoch 200/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 627.6224 - val_loss: 21732.1211
Epoch 201/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 634.1149 - val_loss: 21950.1621
Epoch 202/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 651.7416 - val_loss: 22140.1719
Epoch 203/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 661.7108 - val_loss: 22

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 351.0870 - val_loss: 20264.5059
Epoch 258/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 337.9957 - val_loss: 20246.4785
Epoch 259/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 325.3221 - val_loss: 20227.2441
Epoch 260/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 312.9942 - val_loss: 20207.0059
Epoch 261/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 300.9161 - val_loss: 20185.9805
Epoch 262/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 288.9749 - val_loss: 20164.4629
Epoch 263/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 277.0897 - val_loss: 20142.8281
Epoch 264/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 265.2959 - val_loss: 20121.4727
Epoch 265/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 253.7776 - val_loss: 20100.5234
Epoch 266/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 242.7514 - val_loss: 20080.0430
Epoch 267/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 232.3210 - val_loss: 20

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 271.4368 - val_loss: 19852.4844
Epoch 322/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 310.2443 - val_loss: 20605.0742
Epoch 323/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 326.2210 - val_loss: 20612.6289
Epoch 324/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 498.3848 - val_loss: 23041.6992
Epoch 325/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 730.6380 - val_loss: 24098.6191
Epoch 326/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1128.8802 - val_loss: 24707.7461
Epoch 327/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1637.4058 - val_loss: 19982.5723
Epoch 328/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1603.1771 - val_loss: 18090.0195
Epoch 329/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1954.2682 - val_loss: 20862.2617
Epoch 330/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1638.3970 - val_loss: 26121.9883
Epoch 331/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1412.0299 - val_lo

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 135.4752 - val_loss: 19229.8398
Epoch 386/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 131.4226 - val_loss: 19298.3496
Epoch 387/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 127.9967 - val_loss: 19293.8809
Epoch 388/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 123.1291 - val_loss: 19220.1523
Epoch 389/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 116.0639 - val_loss: 19091.0664
Epoch 390/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 107.4054 - val_loss: 18926.7441
Epoch 391/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 98.7520 - val_loss: 18749.0469
Epoch 392/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 92.0692 - val_loss: 18578.0059
Epoch 393/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 89.0669 - val_loss: 18429.5371
Epoch 394/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 90.7584 - val_loss: 18314.3340
Epoch 395/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 97.2572 - val_loss: 18237.4

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 198.2459 - val_loss: 19986.0508
Epoch 450/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 226.2853 - val_loss: 20195.6113
Epoch 451/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 251.1104 - val_loss: 19945.3164
Epoch 452/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 243.2082 - val_loss: 19368.2949
Epoch 453/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 202.7940 - val_loss: 18689.4316
Epoch 454/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 156.6487 - val_loss: 18107.2148
Epoch 455/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 127.9166 - val_loss: 17724.7148
Epoch 456/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 129.3596 - val_loss: 17549.5039
Epoch 457/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 155.3586 - val_loss: 17533.1289
Epoch 458/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 192.7108 - val_loss: 17593.6992
Epoch 459/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 225.1126 - val_loss: 17

In [35]:
if predictions.ndim != 1:
    predictions = predictions.flatten()

# Check if 'y_test' is not 1-dimensional and reshape if necessary
if y_test.ndim != 1:
    y_test = y_test.flatten()
    
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
print(results)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(mae, 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

     Prediction      Actual
0    173.021469  119.793729
1    431.656281  359.563884
2    235.781036  168.934525
3    410.127228  355.505368
4    522.904480  433.151080
..          ...         ...
112  700.375305  487.957347
113  315.046234  216.123044
114  433.271088  425.953734
115  446.534882  436.229287
116  337.378448  388.424770

[117 rows x 2 columns]
Mean Absolute Error (MAE): 88.22351743665284
Mean Squared Error (MSE): 14100.88647660948
Root Mean Squared Error (RMSE): 118.74715355160932
R-squared (R²): 0.7503762804465625
Mean Absolute Error: 88.22 degrees.
Accuracy: 66.54 %.


C:\Users\EJ-PC\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [36]:
# build model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='swish', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='swish'),
    tf.keras.layers.Dense(128, activation='swish'),
    tf.keras.layers.Dense(1)  # No activation for regression
])

# compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# train model
history = model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_split=0.15)

# evaluate the model
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

#predict
predictions = model.predict(X_test)

Epoch 1/1000


C:\Users\EJ-PC\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 220468.2500 - val_loss: 227412.0000
Epoch 2/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 196888.6719 - val_loss: 140081.0000
Epoch 3/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 110460.2656 - val_loss: 84495.8516
Epoch 4/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 81843.7188 - val_loss: 67447.5859
Epoch 5/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 70389.3750 - val_loss: 61188.0938
Epoch 6/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 63800.0898 - val_loss: 54831.9297
Epoch 7/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 57860.0078 - val_loss: 50116.5859
Epoch 8/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 53193.1094 - val_loss: 45752.3984
Epoch 9/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 49133.0469 - val_loss: 42573.5859
Epoch 10/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 45909.4453 - val_loss: 39902.3203
Epoch 11/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 43118.

Epoch 65/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5637.7949 - val_loss: 18159.3203
Epoch 66/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5493.2256 - val_loss: 18115.3086
Epoch 67/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5346.5142 - val_loss: 18076.9336
Epoch 68/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5204.8105 - val_loss: 18058.0430
Epoch 69/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5068.6426 - val_loss: 18012.3926
Epoch 70/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4931.4131 - val_loss: 18003.1152
Epoch 71/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4803.0283 - val_loss: 17958.0117
Epoch 72/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4673.4863 - val_loss: 17951.4883
Epoch 73/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4552.9561 - val_loss: 17912.0273
Epoch 74/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4435.5645 - val_loss: 17898.9961
Epoch 75/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss:

Epoch 129/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1683.8552 - val_loss: 18902.2500
Epoch 130/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1671.1898 - val_loss: 18341.5586
Epoch 131/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1649.9620 - val_loss: 17732.9688
Epoch 132/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1669.4241 - val_loss: 17209.9746
Epoch 133/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1741.0798 - val_loss: 17044.9121
Epoch 134/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1904.1838 - val_loss: 17521.5938
Epoch 135/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2101.0056 - val_loss: 18844.9160
Epoch 136/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2265.8298 - val_loss: 20549.9727
Epoch 137/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2376.8674 - val_loss: 21479.6816
Epoch 138/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2426.9841 - val_loss: 21453.8281
Epoch 139/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/s

...

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 433.4131 - val_loss: 18530.4492
Epoch 705/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 500.1879 - val_loss: 17894.9160
Epoch 706/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 447.0450 - val_loss: 16077.3652
Epoch 707/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 403.8604 - val_loss: 16289.2969
Epoch 708/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 404.6848 - val_loss: 16883.7012
Epoch 709/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 397.8524 - val_loss: 17612.5449
Epoch 710/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 394.9596 - val_loss: 17452.6719
Epoch 711/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 387.1017 - val_loss: 17248.5684
Epoch 712/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 354.3567 - val_loss: 17591.5000
Epoch 713/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 284.9800 - val_loss: 18575.7227
Epoch 714/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 333.4582 - va

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 428.2694 - val_loss: 18266.2695
Epoch 769/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 480.8695 - val_loss: 16990.1992
Epoch 770/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 373.0062 - val_loss: 16470.0781
Epoch 771/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 442.1078 - val_loss: 16308.1699
Epoch 772/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 506.8301 - val_loss: 17355.4375
Epoch 773/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 520.5087 - val_loss: 17098.9336
Epoch 774/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 431.9922 - val_loss: 17324.2441
Epoch 775/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 369.8263 - val_loss: 17658.5801
Epoch 776/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 331.4826 - val_loss: 18614.9473
Epoch 777/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 383.7195 - val_loss: 18991.1660
Epoch 778/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 421.4573 - va

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 354.1166 - val_loss: 15799.6113
Epoch 833/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 346.9733 - val_loss: 17013.6816
Epoch 834/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 393.0960 - val_loss: 17431.6211
Epoch 835/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 363.1092 - val_loss: 17519.2539
Epoch 836/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 327.7211 - val_loss: 18102.6406
Epoch 837/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 307.7648 - val_loss: 18390.6387
Epoch 838/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 291.4566 - val_loss: 18945.8887
Epoch 839/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 350.3795 - val_loss: 18782.6406
Epoch 840/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 443.9500 - val_loss: 17531.0273
Epoch 841/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 395.5154 - val_loss: 15901.2861
Epoch 842/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 367.2788 - va

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 349.3715 - val_loss: 18126.1660
Epoch 897/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 332.2845 - val_loss: 18738.4785
Epoch 898/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 297.8495 - val_loss: 19209.0957
Epoch 899/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 335.3484 - val_loss: 18978.8008
Epoch 900/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 406.6027 - val_loss: 17619.3242
Epoch 901/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 393.5913 - val_loss: 15913.9326
Epoch 902/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 389.5718 - val_loss: 15923.5107
Epoch 903/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 357.6156 - val_loss: 16816.4277
Epoch 904/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 398.5543 - val_loss: 17658.7773
Epoch 905/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 365.9699 - val_loss: 17868.7891
Epoch 906/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 339.8826 - va

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 333.3722 - val_loss: 16246.9043
Epoch 961/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 308.5583 - val_loss: 16618.1836
Epoch 962/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 317.1962 - val_loss: 17249.3750
Epoch 963/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 297.4847 - val_loss: 17630.0586
Epoch 964/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 296.6090 - val_loss: 17998.8145
Epoch 965/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 292.5820 - val_loss: 18642.9590
Epoch 966/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 296.9469 - val_loss: 19261.0996
Epoch 967/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 300.3342 - val_loss: 18820.9102
Epoch 968/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 327.8391 - val_loss: 17934.6367
Epoch 969/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 326.2622 - val_loss: 16663.2559
Epoch 970/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 283.1545 - va

In [37]:
if predictions.ndim != 1:
    predictions = predictions.flatten()

# Check if 'y_test' is not 1-dimensional and reshape if necessary
if y_test.ndim != 1:
    y_test = y_test.flatten()
    
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
print(results)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(mae, 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

     Prediction      Actual
0    128.733978  119.793729
1    325.666534  359.563884
2    197.955963  168.934525
3    329.711975  355.505368
4    465.542664  433.151080
..          ...         ...
112  565.056458  487.957347
113  306.676605  216.123044
114  448.708008  425.953734
115  342.335693  436.229287
116  476.988251  388.424770

[117 rows x 2 columns]
Mean Absolute Error (MAE): 73.54954778944706
Mean Squared Error (MSE): 10209.076199913401
Root Mean Squared Error (RMSE): 101.03997327747767
R-squared (R²): 0.8192718182325499
Mean Absolute Error: 73.55 degrees.
Accuracy: 77.9 %.


C:\Users\EJ-PC\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [38]:
# build model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='swish', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='swish'),
    tf.keras.layers.Dense(128, activation='swish'),
    tf.keras.layers.Dense(128, activation='swish'),
    tf.keras.layers.Dense(1)  # No activation for regression
])

# compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# train model
history = model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_split=0.15)

# evaluate the model
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

#predict
predictions = model.predict(X_test)

Epoch 1/1000


C:\Users\EJ-PC\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 220129.4844 - val_loss: 217179.9688
Epoch 2/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 169696.8906 - val_loss: 104227.6641
Epoch 3/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 92959.0703 - val_loss: 64359.9492
Epoch 4/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 67800.3672 - val_loss: 54339.7812
Epoch 5/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 56728.6016 - val_loss: 46250.2422
Epoch 6/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 48266.5078 - val_loss: 41627.1992
Epoch 7/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 42369.3828 - val_loss: 38030.3828
Epoch 8/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 37674.6523 - val_loss: 35223.5234
Epoch 9/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 33810.6602 - val_loss: 32821.1641
Epoch 10/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 30539.1172 - val_loss: 30586.8281
Epoch 11/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 27641.3

Epoch 65/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2141.9192 - val_loss: 15911.1455
Epoch 66/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2048.3032 - val_loss: 15962.9404
Epoch 67/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1985.9722 - val_loss: 15958.7715
Epoch 68/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1921.1079 - val_loss: 16145.8750
Epoch 69/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1916.3645 - val_loss: 16305.6973
Epoch 70/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2038.1981 - val_loss: 16758.3281
Epoch 71/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2025.0643 - val_loss: 16890.1406
Epoch 72/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2383.2410 - val_loss: 18110.9082
Epoch 73/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2317.0073 - val_loss: 19009.3945
Epoch 74/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2987.7722 - val_loss: 21508.9316
Epoch 75/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss:

Epoch 129/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1027.5057 - val_loss: 18216.4023
Epoch 130/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1072.7816 - val_loss: 18108.1445
Epoch 131/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1172.1185 - val_loss: 17863.4805
Epoch 132/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1196.7476 - val_loss: 18482.2988
Epoch 133/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1318.8516 - val_loss: 20757.0742
Epoch 134/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2111.8296 - val_loss: 23119.7617
Epoch 135/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3308.9519 - val_loss: 21680.2695
Epoch 136/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3158.3560 - val_loss: 19730.3477
Epoch 137/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2721.3342 - val_loss: 20109.1973
Epoch 138/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2582.8652 - val_loss: 17148.3535
Epoch 139/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/s

...

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 455.6603 - val_loss: 16485.8125
Epoch 705/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 385.3515 - val_loss: 17843.9961
Epoch 706/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 487.1690 - val_loss: 17175.9473
Epoch 707/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 582.4043 - val_loss: 16220.9346
Epoch 708/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 379.0128 - val_loss: 15668.4561
Epoch 709/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 413.5135 - val_loss: 16426.5176
Epoch 710/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 611.0898 - val_loss: 15648.9092
Epoch 711/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 503.2400 - val_loss: 15939.1982
Epoch 712/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 382.7857 - val_loss: 17705.7539
Epoch 713/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 476.1743 - val_loss: 17321.2109
Epoch 714/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 584.8464 - va

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 358.4778 - val_loss: 16382.4980
Epoch 769/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 438.1700 - val_loss: 15768.7656
Epoch 770/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 308.8352 - val_loss: 16554.5840
Epoch 771/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 324.7402 - val_loss: 16479.8906
Epoch 772/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 442.1696 - val_loss: 15676.7578
Epoch 773/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 352.3114 - val_loss: 14853.5918
Epoch 774/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 296.6902 - val_loss: 15701.4092
Epoch 775/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 332.0763 - val_loss: 16516.7305
Epoch 776/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 431.3652 - val_loss: 15784.3691
Epoch 777/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 361.1485 - val_loss: 16321.7627
Epoch 778/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 292.9042 - va

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 397.9999 - val_loss: 14941.6973
Epoch 833/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 384.3428 - val_loss: 15674.2363
Epoch 834/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 493.7491 - val_loss: 17618.7461
Epoch 835/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 506.1085 - val_loss: 17352.8008
Epoch 836/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 433.8182 - val_loss: 17283.1777
Epoch 837/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 358.0359 - val_loss: 17725.3320
Epoch 838/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 548.2130 - val_loss: 17336.0762
Epoch 839/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 534.8224 - val_loss: 16272.8926
Epoch 840/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 329.0190 - val_loss: 15339.9346
Epoch 841/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 471.8024 - val_loss: 16263.4238
Epoch 842/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 497.7558 - va

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 268.6399 - val_loss: 16127.2627
Epoch 897/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 327.0447 - val_loss: 15827.9668
Epoch 898/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 452.0995 - val_loss: 16705.4336
Epoch 899/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 357.4993 - val_loss: 17442.7773
Epoch 900/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 329.1364 - val_loss: 18304.5039
Epoch 901/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 504.8089 - val_loss: 16982.2383
Epoch 902/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 341.1137 - val_loss: 15780.8223
Epoch 903/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 297.9475 - val_loss: 15434.7246
Epoch 904/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 414.7273 - val_loss: 16686.5059
Epoch 905/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 451.1089 - val_loss: 16806.5430
Epoch 906/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 316.0681 - va

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 268.6615 - val_loss: 15947.8379
Epoch 961/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 334.3255 - val_loss: 16033.8926
Epoch 962/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 448.6153 - val_loss: 14879.5420
Epoch 963/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 251.0726 - val_loss: 15107.0176
Epoch 964/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 232.5829 - val_loss: 15555.7549
Epoch 965/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 343.0634 - val_loss: 15536.3096
Epoch 966/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 353.9874 - val_loss: 14934.5527
Epoch 967/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 225.3873 - val_loss: 16023.3408
Epoch 968/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 353.3315 - val_loss: 15450.5938
Epoch 969/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 374.6342 - val_loss: 14113.5889
Epoch 970/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 245.5255 - va

In [39]:
if predictions.ndim != 1:
    predictions = predictions.flatten()

# Check if 'y_test' is not 1-dimensional and reshape if necessary
if y_test.ndim != 1:
    y_test = y_test.flatten()
    
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
print(results)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(mae, 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

     Prediction      Actual
0     85.044647  119.793729
1    384.532318  359.563884
2    145.187149  168.934525
3    325.587524  355.505368
4    452.441467  433.151080
..          ...         ...
112  654.096130  487.957347
113  438.659882  216.123044
114  453.261200  425.953734
115  375.102692  436.229287
116  354.726532  388.424770

[117 rows x 2 columns]
Mean Absolute Error (MAE): 64.22054326315897
Mean Squared Error (MSE): 8484.90409812794
Root Mean Squared Error (RMSE): 92.11353916839771
R-squared (R²): 0.8497943143828375
Mean Absolute Error: 64.22 degrees.
Accuracy: 79.05 %.


C:\Users\EJ-PC\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [40]:
# build model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='swish', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='swish'),
    tf.keras.layers.Dense(128, activation='swish'),
    tf.keras.layers.Dense(128, activation='swish'),
    tf.keras.layers.Dense(1)  # No activation for regression
])

# compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# train model
history = model.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.15)

# evaluate the model
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

#predict
predictions = model.predict(X_test)

Epoch 1/500


C:\Users\EJ-PC\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 222721.3906 - val_loss: 228577.3750
Epoch 2/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 217077.3594 - val_loss: 204646.5781
Epoch 3/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 181036.4531 - val_loss: 146709.3594
Epoch 4/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 116160.3281 - val_loss: 86491.2578
Epoch 5/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 84094.0547 - val_loss: 64606.8320
Epoch 6/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 71720.2891 - val_loss: 59226.7617
Epoch 7/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 64015.7383 - val_loss: 54170.2344
Epoch 8/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 57457.9727 - val_loss: 48093.0430
Epoch 9/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 51434.5859 - val_loss: 43455.5156
Epoch 10/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 46468.6758 - val_loss: 39463.1914
Epoch 11/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 42146.8125 - 

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5347.6885 - val_loss: 20032.6504
Epoch 66/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4855.5117 - val_loss: 19844.6738
Epoch 67/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4587.5762 - val_loss: 19935.9316
Epoch 68/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4339.4604 - val_loss: 20253.6426
Epoch 69/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4132.0645 - val_loss: 20751.9746
Epoch 70/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3988.3525 - val_loss: 21354.4062
Epoch 71/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3901.4907 - val_loss: 21990.6504
Epoch 72/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3854.7056 - val_loss: 22610.2910
Epoch 73/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3830.1809 - val_loss: 23177.7285
Epoch 74/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 3812.9333 - val_loss: 23668.5293
Epoch 75/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3793.3223 - val_loss: 2

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 965.4058 - val_loss: 18396.7988
Epoch 130/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 940.0994 - val_loss: 18658.4902
Epoch 131/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 912.2402 - val_loss: 19034.9688
Epoch 132/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 890.3521 - val_loss: 19519.3887
Epoch 133/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 883.6843 - val_loss: 20092.0645
Epoch 134/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 897.8009 - val_loss: 20719.5449
Epoch 135/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 933.6748 - val_loss: 21359.6328
Epoch 136/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 989.2953 - val_loss: 21960.0020
Epoch 137/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1060.3395 - val_loss: 22455.1816
Epoch 138/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1139.2039 - val_loss: 22776.9902
Epoch 139/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1214.8125 - val_loss:

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1878.4907 - val_loss: 17005.4688
Epoch 194/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1817.8070 - val_loss: 18127.3105
Epoch 195/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1426.2739 - val_loss: 21140.5254
Epoch 196/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1288.9716 - val_loss: 25404.2598
Epoch 197/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1766.4908 - val_loss: 27361.5527
Epoch 198/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2378.5598 - val_loss: 23703.1426
Epoch 199/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1932.7759 - val_loss: 18494.9551
Epoch 200/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1082.5635 - val_loss: 16122.5000
Epoch 201/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1078.7776 - val_loss: 15771.6924
Epoch 202/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1385.6355 - val_loss: 16179.1562
Epoch 203/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1273.9884 - v

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 714.8956 - val_loss: 15953.3232
Epoch 258/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 692.9810 - val_loss: 17169.9004
Epoch 259/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 501.5237 - val_loss: 19527.3262
Epoch 260/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 463.8589 - val_loss: 21723.7129
Epoch 261/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 656.4208 - val_loss: 21796.8633
Epoch 262/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 777.4836 - val_loss: 19480.4824
Epoch 263/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 556.3235 - val_loss: 16980.4258
Epoch 264/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 383.2357 - val_loss: 15819.1045
Epoch 265/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 519.1176 - val_loss: 15697.6611
Epoch 266/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 666.7057 - val_loss: 16251.3701
Epoch 267/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 588.4400 - val_loss: 17

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 616.0495 - val_loss: 16823.5293
Epoch 322/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 374.8596 - val_loss: 15409.2627
Epoch 323/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 498.4806 - val_loss: 15330.8936
Epoch 324/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 681.9285 - val_loss: 15960.9092
Epoch 325/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 597.0845 - val_loss: 17711.0078
Epoch 326/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 444.4510 - val_loss: 20485.2656
Epoch 327/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 545.6473 - val_loss: 21862.6895
Epoch 328/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 807.9070 - val_loss: 19888.0918
Epoch 329/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 678.4114 - val_loss: 16780.5078
Epoch 330/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 404.7786 - val_loss: 15321.8174
Epoch 331/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 540.5964 - val_loss: 15

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 602.8649 - val_loss: 15574.5322
Epoch 386/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 570.2800 - val_loss: 17098.3809
Epoch 387/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 405.9432 - val_loss: 19826.2656
Epoch 388/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 475.9487 - val_loss: 21129.5684
Epoch 389/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 715.5432 - val_loss: 18884.7363
Epoch 390/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 546.1316 - val_loss: 16074.2842
Epoch 391/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 337.4641 - val_loss: 15130.1572
Epoch 392/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 521.9590 - val_loss: 15299.5986
Epoch 393/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 595.2010 - val_loss: 16351.7549
Epoch 394/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 440.0560 - val_loss: 18738.7637
Epoch 395/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 396.3548 - val_loss: 20

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 363.4171 - val_loss: 15080.1211
Epoch 450/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 501.6577 - val_loss: 15715.8486
Epoch 451/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 400.2056 - val_loss: 17551.5430
Epoch 452/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 317.2673 - val_loss: 19828.2734
Epoch 453/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 454.1409 - val_loss: 19575.5723
Epoch 454/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 552.2027 - val_loss: 16800.0566
Epoch 455/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 302.9203 - val_loss: 15249.5049
Epoch 456/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 341.3553 - val_loss: 15066.2998
Epoch 457/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 480.6510 - val_loss: 15688.9326
Epoch 458/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 401.1738 - val_loss: 17553.5410
Epoch 459/500
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 308.0542 - val_loss: 19

In [41]:
if predictions.ndim != 1:
    predictions = predictions.flatten()

# Check if 'y_test' is not 1-dimensional and reshape if necessary
if y_test.ndim != 1:
    y_test = y_test.flatten()
    
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
print(results)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(mae, 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

     Prediction      Actual
0    158.266556  119.793729
1    369.913208  359.563884
2    301.869202  168.934525
3    401.196533  355.505368
4    584.475891  433.151080
..          ...         ...
112  672.909363  487.957347
113  306.458496  216.123044
114  413.199005  425.953734
115  385.870331  436.229287
116  335.627502  388.424770

[117 rows x 2 columns]
Mean Absolute Error (MAE): 79.56523247485332
Mean Squared Error (MSE): 12061.24712034904
Root Mean Squared Error (RMSE): 109.82370928150733
R-squared (R²): 0.7864833977899914
Mean Absolute Error: 79.57 degrees.
Accuracy: 71.45 %.


C:\Users\EJ-PC\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
